### This notebook contains codes and functions to scrape google reviews from a provided restaurant URL:

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
%run scrape_googlemaps_functions.ipynb

In [4]:
from selenium import webdriver
# options = Options()
# options.add_argument("--lang=en")
# driver = webdriver.Chrome(chrome_options=options)

driver = webdriver.Chrome(executable_path=r'chromedriver_win32\chromedriver.exe')


In [5]:
import pandas as pd
allRests = pd.read_csv('data/restaurant_info_Toronto_tripAdvisor_v2.csv')
allRests


,Unnamed: 0,Unnamed: 0.1,name,stars,cuisine,retrieve-date,name2,json,name_google_0,place_id_0,...,rating_google_0,name_google_1,place_id_1,formatted_address_1,business_status_1,lat_1,lng_1,rating_google_1,url_0,url_1
0,0,0,1. Goodfellas Wood Oven Pizza,4.5 of 5 bubbles,"Italian, Pizza",2021-08-09 19:15:40.484368-04:00,Goodfellas Wood Oven Pizza,"[{'business_status': 'OPERATIONAL', 'formatted...",Goodfellas Wood Oven Pizza,ChIJedJ0GGI2K4gR6dco_CjRntY,...,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
1,1,1,2. Moxie's Grill & Bar,4.5 of 5 bubbles,"Grill, Bar",2021-08-09 19:15:40.732702-04:00,Moxie's Grill & Bar,"[{'business_status': 'OPERATIONAL', 'formatted...",Moxie's Grill & Bar,ChIJfQJkpzMyK4gRDnUt3YFXa5w,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
2,2,2,3. New Orleans Seafood & Steakhouse,4.5 of 5 bubbles,"Steakhouse, Cajun & Creole",2021-08-09 19:15:40.732702-04:00,New Orleans Seafood & Steakhouse,"[{'business_status': 'OPERATIONAL', 'formatted...",New Orleans Seafood & Steakhouse,ChIJ466I-9o2K4gRxzeisSIQUzg,...,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
3,3,3,4. Scaramouche Restaurant,4.5 of 5 bubbles,"French, International",2021-08-09 19:15:40.732702-04:00,Scaramouche Restaurant,"[{'business_status': 'OPERATIONAL', 'formatted...",Scaramouche Restaurant,ChIJrxCATO8zK4gRHe57JQWul2Q,...,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
4,4,4,5. ALO RESTAURANT,4.5 of 5 bubbles,"French, European",2021-08-09 19:15:40.732702-04:00,ALO RESTAURANT,"[{'business_status': 'OPERATIONAL', 'formatted...",Alo,ChIJryijc9s0K4gRG9aU7SDTXdA,...,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,2095,2095,2096. The Burren Pub,4.5 of 5 bubbles,Pub,2021-08-09 19:23:06.617804-04:00,The Burren Pub,"[{'business_status': 'OPERATIONAL', 'formatted...",The Burren Pub,ChIJeT8ioLPL1IkRNS2ses5dOUY,...,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
1976,2096,2096,2097. The Beaver Cafe,4.0 of 5 bubbles,"Bar, Cafe",2021-08-09 19:23:06.617804-04:00,The Beaver Cafe,"[{'business_status': 'CLOSED_PERMANENTLY', 'fo...",Beaver Cafe,ChIJBx_Gvao1K4gRrFGEV1E0Ejs,...,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
1977,2097,2097,2098. Tot Cat Cafe,3.5 of 5 bubbles,Cafe,2021-08-09 19:23:06.617804-04:00,Tot Cat Cafe,"[{'business_status': 'CLOSED_PERMANENTLY', 'fo...",Tot The Cat Cafe,ChIJAc4ay8E0K4gRlFr93DwkSdY,...,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN
1978,2098,2098,2099. Gatto Wood Oven Pizza,4.0 of 5 bubbles,"Italian, Pizza",2021-08-09 19:23:06.617804-04:00,Gatto Wood Oven Pizza,"[{'business_status': 'OPERATIONAL', 'formatted...",GATTO Wood Oven Pizza,ChIJReU0Gyo3K4gRh1aes3ounbE,...,4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,NaN


In [6]:
allRests['url_0'].isnull().any()

False

In [6]:
# # allRests2 = allRests[8:9].copy()
# # len(allRests2)
# # # import os
# # ~ os.path.exists(csvfile)
# not os.path.exists(csvfile)
allRests.loc[147]['json']
# allRests.loc[153]['url_0']

"[{'business_status': 'OPERATIONAL', 'formatted_address': '895 Yonge St 2nd Floor, Toronto, ON M4W 2H2, Canada', 'geometry': {'location': {'lat': 43.6747212, 'lng': -79.38835019999999}, 'viewport': {'northeast': {'lat': 43.67607102989272, 'lng': -79.38700037010727}, 'southwest': {'lat': 43.67337137010728, 'lng': -79.38970002989272}}}, 'name': 'Golden Thai Health Care', 'place_id': 'ChIJaSTAO6U1K4gR0zkXOp_y2zs', 'rating': 0}]"

In [9]:
import os
t0 = time.time()
t1 = [t0]
N = 200
i = 0
# allRests2 = allRests[49:].copy()
for index,rest in allRests.iterrows():
    url = rest['url_0']
    csvfile = 'data/reviews/'+rest['place_id_0']+'.csv'
    if not os.path.isfile(csvfile):
        driver.get(url)
        time.sleep(2)

        review_df, num  = get_reviews(driver,N)
        review_df['num_reviews_total'] = num
        review_df.to_csv(csvfile) 
    else:
        temp = pd.read_csv(csvfile)
        try:
            num = temp['num_reviews_total'][0]
        except Exception as e:
            num = 0
    
    t1.append(time.time())
    i+=1
    print(f'Scraped {min(num,N)} reviews from restaurant {index} in {round((t1[i]-t1[i-1])/60,2)} minutes...')

print(t1)

Scraped 200 reviews from restaurant 0 in 0.0 minutes...
Scraped 200 reviews from restaurant 1 in 0.0 minutes...
Scraped 200 reviews from restaurant 2 in 0.0 minutes...
Scraped 200 reviews from restaurant 3 in 0.0 minutes...
Scraped 200 reviews from restaurant 4 in 0.0 minutes...
Scraped 200 reviews from restaurant 5 in 0.0 minutes...
Scraped 200 reviews from restaurant 6 in 0.0 minutes...
Scraped 200 reviews from restaurant 7 in 0.0 minutes...
Scraped 200 reviews from restaurant 8 in 0.0 minutes...
Scraped 200 reviews from restaurant 9 in 0.0 minutes...
Scraped 2 reviews from restaurant 10 in 0.0 minutes...
Scraped 84 reviews from restaurant 11 in 0.0 minutes...
Scraped 200 reviews from restaurant 12 in 0.0 minutes...
Scraped 200 reviews from restaurant 13 in 0.0 minutes...
Scraped 200 reviews from restaurant 14 in 0.0 minutes...
Scraped 200 reviews from restaurant 15 in 0.0 minutes...
Scraped 200 reviews from restaurant 16 in 0.0 minutes...
Scraped 200 reviews from restaurant 17 in 0.

Scraped 200 reviews from restaurant 162 in 0.0 minutes...
Scraped 200 reviews from restaurant 163 in 0.0 minutes...
Scraped 200 reviews from restaurant 164 in 0.0 minutes...
Scraped 200 reviews from restaurant 165 in 0.0 minutes...
Scraped 200 reviews from restaurant 166 in 0.0 minutes...
Scraped 200 reviews from restaurant 167 in 0.0 minutes...
Scraped 200 reviews from restaurant 168 in 0.0 minutes...
Scraped 200 reviews from restaurant 169 in 0.0 minutes...
Scraped 200 reviews from restaurant 170 in 0.0 minutes...
Scraped 200 reviews from restaurant 171 in 0.0 minutes...
Scraped 200 reviews from restaurant 172 in 0.0 minutes...
Scraped 200 reviews from restaurant 173 in 0.0 minutes...
Scraped 7 reviews from restaurant 174 in 0.0 minutes...
Scraped 200 reviews from restaurant 175 in 0.0 minutes...
Scraped 200 reviews from restaurant 176 in 0.0 minutes...
Scraped 200 reviews from restaurant 177 in 0.0 minutes...
Scraped 101 reviews from restaurant 178 in 0.0 minutes...
Scraped 200 revi

Scraped 200 reviews from restaurant 307 in 0.0 minutes...
Scraped 200 reviews from restaurant 308 in 0.0 minutes...
Scraped 200 reviews from restaurant 309 in 0.0 minutes...
Scraped 200 reviews from restaurant 310 in 0.0 minutes...
Scraped 200 reviews from restaurant 311 in 0.0 minutes...
Scraped 200 reviews from restaurant 312 in 0.0 minutes...
Scraped 200 reviews from restaurant 313 in 0.0 minutes...
Scraped 200 reviews from restaurant 314 in 0.0 minutes...
Scraped 200 reviews from restaurant 315 in 0.0 minutes...
Scraped 200 reviews from restaurant 316 in 0.0 minutes...
Scraped 200 reviews from restaurant 317 in 0.0 minutes...
Scraped 200 reviews from restaurant 318 in 0.0 minutes...
Scraped 200 reviews from restaurant 319 in 0.0 minutes...
Scraped 200 reviews from restaurant 320 in 0.0 minutes...
Scraped 200 reviews from restaurant 321 in 0.0 minutes...
Scraped 200 reviews from restaurant 322 in 0.0 minutes...
Scraped 200 reviews from restaurant 323 in 0.0 minutes...
Scraped 200 re

Scraped 200 reviews from restaurant 457 in 0.0 minutes...
Scraped 136 reviews from restaurant 458 in 0.0 minutes...
Scraped 200 reviews from restaurant 459 in 0.0 minutes...
Scraped 200 reviews from restaurant 460 in 0.0 minutes...
Scraped 200 reviews from restaurant 461 in 0.0 minutes...
Scraped 200 reviews from restaurant 462 in 0.0 minutes...
Scraped 200 reviews from restaurant 463 in 0.0 minutes...
Scraped 200 reviews from restaurant 464 in 0.0 minutes...
Scraped 200 reviews from restaurant 465 in 0.0 minutes...
Scraped 200 reviews from restaurant 466 in 0.0 minutes...
Scraped 50 reviews from restaurant 467 in 0.0 minutes...
Scraped 200 reviews from restaurant 468 in 0.0 minutes...
Scraped 200 reviews from restaurant 469 in 0.0 minutes...
Scraped 50 reviews from restaurant 470 in 0.0 minutes...
Scraped 0 reviews from restaurant 471 in 0.0 minutes...
Scraped 200 reviews from restaurant 472 in 0.0 minutes...
Scraped 200 reviews from restaurant 473 in 0.0 minutes...
Scraped 5 reviews 

Scraped 200 reviews from restaurant 614 in 0.0 minutes...
Scraped 200 reviews from restaurant 615 in 0.0 minutes...
Scraped 200 reviews from restaurant 616 in 0.0 minutes...
Scraped 200 reviews from restaurant 617 in 0.0 minutes...
Scraped 200 reviews from restaurant 618 in 0.0 minutes...
Scraped 200 reviews from restaurant 619 in 0.0 minutes...
Scraped 200 reviews from restaurant 620 in 0.0 minutes...
Scraped 190 reviews from restaurant 621 in 0.0 minutes...
Scraped 200 reviews from restaurant 622 in 0.0 minutes...
Scraped 200 reviews from restaurant 623 in 0.0 minutes...
Scraped 200 reviews from restaurant 624 in 0.0 minutes...
Scraped 200 reviews from restaurant 625 in 0.0 minutes...
Scraped 200 reviews from restaurant 626 in 0.0 minutes...
Scraped 200 reviews from restaurant 627 in 0.0 minutes...
Scraped 200 reviews from restaurant 628 in 0.0 minutes...
Scraped 200 reviews from restaurant 629 in 0.0 minutes...
Scraped 198 reviews from restaurant 630 in 0.0 minutes...
Scraped 200 re

...

Scraped 200 reviews from restaurant 1372 in 2.2 minutes...
Scraped 26 reviews from restaurant 1373 in 0.0 minutes...
Scraped 200 reviews from restaurant 1374 in 0.0 minutes...
Scraped 200 reviews from restaurant 1375 in 0.0 minutes...
Scraped 200 reviews from restaurant 1376 in 0.0 minutes...
Scraped 184 reviews from restaurant 1377 in 0.0 minutes...
Scraped 68 reviews from restaurant 1378 in 0.0 minutes...
Scraped 159 reviews from restaurant 1379 in 0.0 minutes...
Scraped 200 reviews from restaurant 1380 in 0.0 minutes...
Scraped 200 reviews from restaurant 1381 in 0.0 minutes...
Scraped 180 reviews from restaurant 1382 in 0.0 minutes...
Scraped 200 reviews from restaurant 1383 in 0.0 minutes...
Scraped 97 reviews from restaurant 1384 in 0.0 minutes...
Scraped 200 reviews from restaurant 1385 in 0.0 minutes...
Scraped 200 reviews from restaurant 1386 in 0.0 minutes...
Scraped 200 reviews from restaurant 1387 in 0.0 minutes...
Scraped 200 reviews from restaurant 1388 in 0.0 minutes...


Scraped 200 reviews from restaurant 1530 in 0.0 minutes...
Scraped 200 reviews from restaurant 1531 in 0.0 minutes...
Scraped 200 reviews from restaurant 1532 in 0.0 minutes...
Scraped 200 reviews from restaurant 1533 in 0.0 minutes...
Scraped 200 reviews from restaurant 1534 in 0.0 minutes...
Scraped 200 reviews from restaurant 1535 in 0.0 minutes...
Scraped 200 reviews from restaurant 1536 in 0.0 minutes...
Scraped 200 reviews from restaurant 1537 in 0.0 minutes...
Scraped 200 reviews from restaurant 1538 in 0.0 minutes...
Scraped 200 reviews from restaurant 1539 in 0.0 minutes...
Scraped 200 reviews from restaurant 1540 in 0.0 minutes...
Scraped 200 reviews from restaurant 1541 in 0.0 minutes...
Scraped 200 reviews from restaurant 1542 in 0.0 minutes...
Scraped 185 reviews from restaurant 1543 in 0.0 minutes...
Scraped 200 reviews from restaurant 1544 in 0.0 minutes...
Scraped 200 reviews from restaurant 1545 in 0.0 minutes...
Scraped 200 reviews from restaurant 1546 in 0.0 minutes.

Scraped 200 reviews from restaurant 1670 in 0.0 minutes...
Scraped 200 reviews from restaurant 1671 in 0.0 minutes...
Scraped 0 reviews from restaurant 1672 in 0.0 minutes...
Scraped 31 reviews from restaurant 1673 in 0.59 minutes...
Scraped 181 reviews from restaurant 1674 in 0.0 minutes...
Scraped 0 reviews from restaurant 1675 in 0.0 minutes...
Scraped 200 reviews from restaurant 1676 in 0.0 minutes...
Scraped 68 reviews from restaurant 1677 in 0.0 minutes...
Scraped 200 reviews from restaurant 1678 in 0.0 minutes...
Scraped 200 reviews from restaurant 1679 in 0.0 minutes...
Scraped 200 reviews from restaurant 1680 in 0.0 minutes...
Scraped 110 reviews from restaurant 1681 in 0.0 minutes...
Scraped 40 reviews from restaurant 1682 in 0.0 minutes...
Scraped 200 reviews from restaurant 1683 in 0.0 minutes...
Scraped 200 reviews from restaurant 1684 in 0.0 minutes...
Scraped 200 reviews from restaurant 1685 in 0.0 minutes...
Scraped 200 reviews from restaurant 1686 in 0.0 minutes...
Scr

Scraped 200 reviews from restaurant 1814 in 0.0 minutes...
Scraped 43 reviews from restaurant 1815 in 0.0 minutes...
Scraped 200 reviews from restaurant 1816 in 0.0 minutes...
Scraped 131 reviews from restaurant 1817 in 0.0 minutes...
Scraped 119 reviews from restaurant 1818 in 1.49 minutes...
Scraped 196 reviews from restaurant 1819 in 2.25 minutes...
Scraped 200 reviews from restaurant 1820 in 0.0 minutes...
Scraped 142 reviews from restaurant 1821 in 0.0 minutes...
Scraped 200 reviews from restaurant 1822 in 0.0 minutes...
Scraped 0 reviews from restaurant 1823 in 0.0 minutes...
Scraped 200 reviews from restaurant 1824 in 0.0 minutes...
Scraped 200 reviews from restaurant 1825 in 2.47 minutes...
Scraped 200 reviews from restaurant 1826 in 0.0 minutes...
Scraped 200 reviews from restaurant 1827 in 0.0 minutes...
Scraped 200 reviews from restaurant 1828 in 0.0 minutes...
Scraped 200 reviews from restaurant 1829 in 0.0 minutes...
Scraped 200 reviews from restaurant 1830 in 0.0 minutes.

Scraped 200 reviews from restaurant 1958 in 2.35 minutes...
Scraped 137 reviews from restaurant 1959 in 0.0 minutes...
Scraped 200 reviews from restaurant 1960 in 0.0 minutes...
Scraped 200 reviews from restaurant 1961 in 0.0 minutes...
Scraped 200 reviews from restaurant 1962 in 0.0 minutes...
Scraped 6 reviews from restaurant 1963 in 0.0 minutes...
Scraped 200 reviews from restaurant 1964 in 2.4 minutes...
Scraped 200 reviews from restaurant 1965 in 0.0 minutes...
Scraped 200 reviews from restaurant 1966 in 0.0 minutes...
Scraped 60 reviews from restaurant 1967 in 0.0 minutes...
Scraped 200 reviews from restaurant 1968 in 0.0 minutes...
Scraped 200 reviews from restaurant 1969 in 0.0 minutes...
Scraped 200 reviews from restaurant 1970 in 0.0 minutes...
Scraped 200 reviews from restaurant 1971 in 0.0 minutes...
Scraped 114 reviews from restaurant 1972 in 0.0 minutes...
Scraped 200 reviews from restaurant 1973 in 0.0 minutes...
Scraped 129 reviews from restaurant 1974 in 0.0 minutes...

In [11]:
len(t1)

1981

In [12]:
#to add those with second place_ids
t0 = time.time()
t1 = [t0]
N = 200
i = 0
## Run through those first occurences; that's why i flag the last ones as duplicated and first ones to go
for index,rest in allRests[~allRests['place_id_1'].isna()].iterrows():
    url = rest['url_1']
    csvfile = 'data/reviews/'+rest['place_id_1']+'.csv'
    if not os.path.isfile(csvfile):
        driver.get(url)
        time.sleep(2)

        review_df, num  = get_reviews(driver,N)
        review_df['num_reviews_total'] = num
        review_df.to_csv(csvfile) 
    else:
        temp = pd.read_csv(csvfile)
        try:
            num = temp['num_reviews_total'][0]
        except Exception as e:
            num = 0
    
    t1.append(time.time())
    i+=1
    print(f'Scraped {min(num,N)} reviews from restaurant {index} in {round((t1[i]-t1[i-1])/60,2)} minutes...')

print(t1)


Scraped 136 reviews from restaurant 19 in 0.0 minutes...
Scraped 200 reviews from restaurant 24 in 0.0 minutes...
Scraped 200 reviews from restaurant 29 in 0.0 minutes...
Scraped 200 reviews from restaurant 47 in 0.0 minutes...
Scraped 200 reviews from restaurant 48 in 0.0 minutes...
Scraped 200 reviews from restaurant 66 in 0.0 minutes...
Scraped 61 reviews from restaurant 86 in 0.0 minutes...
Scraped 200 reviews from restaurant 103 in 0.0 minutes...
Scraped 200 reviews from restaurant 113 in 0.0 minutes...
Scraped 200 reviews from restaurant 117 in 0.0 minutes...
Scraped 200 reviews from restaurant 125 in 0.0 minutes...
Scraped 200 reviews from restaurant 150 in 0.0 minutes...
Scraped 200 reviews from restaurant 151 in 0.0 minutes...
Scraped 200 reviews from restaurant 158 in 0.0 minutes...
Scraped 172 reviews from restaurant 164 in 0.0 minutes...
Scraped 200 reviews from restaurant 166 in 0.0 minutes...
Scraped 200 reviews from restaurant 176 in 0.0 minutes...
Scraped 200 reviews fr

Scraped 200 reviews from restaurant 1483 in 2.51 minutes...
Scraped 200 reviews from restaurant 1487 in 2.39 minutes...
Scraped 200 reviews from restaurant 1497 in 0.0 minutes...
Scraped 200 reviews from restaurant 1504 in 0.0 minutes...
Scraped 200 reviews from restaurant 1505 in 0.0 minutes...
Scraped 200 reviews from restaurant 1507 in 0.0 minutes...
Scraped 11 reviews from restaurant 1514 in 0.0 minutes...
Scraped 80 reviews from restaurant 1518 in 0.0 minutes...
Scraped 200 reviews from restaurant 1522 in 0.0 minutes...
Scraped 200 reviews from restaurant 1545 in 0.0 minutes...
Scraped 200 reviews from restaurant 1559 in 0.0 minutes...
Scraped 41 reviews from restaurant 1578 in 0.0 minutes...
Scraped 45 reviews from restaurant 1597 in 0.0 minutes...
Scraped 200 reviews from restaurant 1605 in 0.0 minutes...
Scraped 15 reviews from restaurant 1617 in 0.0 minutes...
Scraped 200 reviews from restaurant 1629 in 0.0 minutes...
Scraped 200 reviews from restaurant 1631 in 0.0 minutes...


In [ ]:
# SCROLL_PAUSE_TIME = 0.5

# # Get scroll height
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # Scroll down to bottom
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # Wait to load page
#     time.sleep(SCROLL_PAUSE_TIME)

#     # Calculate new scroll height and compare with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height


In [ ]:
# scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc')
# driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
# time.sleep(2)
# last_height = driver.execute_script("return document.body.scrollHeight")
# print(last_height)

In [ ]:
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
# soup = BeautifulSoup(driver.page_source,'html.parser')
# x = soup.find('button', class_='widget-pane-link')
# x.text

In [ ]:
# <button aria-label="897 reviews" vet="3648" jsaction="pane.rating.moreReviews" jstcache="165" class="widget-pane-link" jsan="7.widget-pane-link,0.aria-label,0.vet,0.jsaction">897 reviews</button>

In [ ]:
# <span jstcache="142" jsinstance="0" class="h0ySl-wcwwM-E70qVe"><span aria-hidden="true" jstcache="143" class="h0ySl-wcwwM-RWgCYc-YPIHXb-divider" style="display:none">·</span><span jsaction="pane.rating.moreReviews" jstcache="144" class="OAO0-ZEhYpd-vJ7A6b OAO0-ZEhYpd-vJ7A6b-qnnXGd" jsan="7.OAO0-ZEhYpd-vJ7A6b,7.OAO0-ZEhYpd-vJ7A6b-qnnXGd,0.jsaction,t-8ok-g490efo"><span jstcache="164" style="color:#1A73E8" jsan="5.color"><button aria-label="1,838 reviews" vet="3648" jsaction="pane.rating.moreReviews" jstcache="165" class="widget-pane-link" jsan="7.widget-pane-link,0.aria-label,0.vet,0.jsaction">1,838 reviews</button><span jstcache="166" style="display:none"></span></span><span jstcache="167"><span jstcache="171" vet="3648" jsan="t-SEZnAsDb2GY,0.vet"><a jstcache="173" style="display:none"></a><a jstcache="174" style="display:none"></a><a jstcache="175" style="display:none"></a></span></span><button jstcache="168" jsaction="pane.clickPhoneNumber" class="widget-pane-link" style="display:none" jsan="7.widget-pane-link,22.jsaction"></button><span jstcache="169" class="widget-pane-link" style="display:none" jsan="7.widget-pane-link"></span></span></span>

In [ ]:

# review_df, num  = get_reviews(driver)
# review_df['num_reviews_total'] = num

In [ ]:
# review_df['review-retrieve-date']

In [ ]:
# links = driver.find_elements_by_xpath('//button[@jsaction=\'pane.reviewlist.goToReviews\']') 
# links = driver.find_elements_by_xpath('//button[@jsaction=\'pane.placeActions.save;keydown:pane.placeActions.save\']') 
# links = driver.find_elements_by_xpath('//button[@jsaction=\'pane.review.expandReview\']') 
# links = driver.find_elements_by_xpath('//span[@jsan=\'7.wNNZR,7.gm2-button-alt\']') 
# links = driver.find_elements_by_xpath('//*[contains(@aria-label,"More reviews")]') 
# links = driver.find_elements_by_xpath('//button[@jsaction=\'pane.rating.moreReviews\']') 

# # <span jstcache="282" class="wNNZR gm2-button-alt" jsan="7.wNNZR,7.gm2-button-alt">More reviews (1,535)</span>
# # <button data-review-id="ChdDSUhNMG9nS0VJQ0FnSUNhNkx6QXRnRRAB" ved="1i:4,t:63707,e:2,p:OIIVYd_eM7az5NoP_p2N8AI:318" aria-label=" See more " jstcache="599" jsaction="pane.review.expandReview" class="ODSEW-KoToPc-ShBeI gXqMYb-hSRGPd" jsan="7.ODSEW-KoToPc-ShBeI,7.gXqMYb-hSRGPd,0.data-review-id,0.ved,0.aria-label,22.jsaction">More</button>
# # links = driver.find_elements_by_xpath('//*[contains(@aria-label="reviews")]') 
# # <span jstcache="282" class="wNNZR gm2-button-alt" jsan="7.wNNZR,7.gm2-button-alt">More reviews (1,535)</span>
# # links = driver.find_elements_by_xpath('//span[@jsan=\'7.wNNZR,7.gm2-button-alt\']')
# # links = driver.find_elements_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[4]/div[2]/div/button') 

# # //*[@id="pane"]/div/div[1]/div/div/div[4]/div[2]/div/button
# # <span jstcache="282" class="wNNZR gm2-button-alt" jsan="7.wNNZR,7.gm2-button-alt">More reviews (1,535)</span>
# # //*[@id="pane"]/div/div[1]/div/div/div[45]/div/div/button
# # links = driver.find_elements_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[45]/div/div/button')

# # <span jstcache="282" class="wNNZR gm2-button-alt" jsan="7.wNNZR,7.gm2-button-alt">More reviews (1,535)</span>
# # <button aria-label="More reviews (1,535)" jsaction="pane.reviewlist.goToReviews" jstcache="280" class="BgrMEd BgrMEd-text" jsan="7.BgrMEd,7.BgrMEd-text,0.aria-label,0.jsaction"><img jstcache="281" style="display:none"><span jstcache="282" class="wNNZR gm2-button-alt" jsan="7.wNNZR,7.gm2-button-alt">More reviews (1,535)</span></button>
# # //*[@id="pane"]/div/div[1]/div/div/div[45]/div/div/button
# # <button aria-label="More reviews (1,535)" jsaction="pane.reviewlist.goToReviews" jstcache="280" class="BgrMEd BgrMEd-text" jsan="7.BgrMEd,7.BgrMEd-text,0.aria-label,0.jsaction"><img jstcache="281" style="display:none"><span jstcache="282" class="wNNZR gm2-button-alt" jsan="7.wNNZR,7.gm2-button-alt">More reviews (1,535)</span></button>
# # #<button ved="1i:1,t:18519,e:0,p:kPkcYIz-Dtql-QaL1YawDw:1969" jstcache="1202" jsaction="pane.reviewChart.moreReviews" class="gm2-button-alt jqnFjrOWMVU__button-blue" jsan="7.gm2-button-alt,7.jqnFjrOWMVU__button-blue,0.ved,22.jsaction">14 reviews</button>
# # #<button aria-label="14 reviews" vet="3648" jsaction="pane.rating.moreReviews" jstcache="1010" class="widget-pane-link" jsan="7.widget-pane-link,0.aria-label,0.vet,0.jsaction">14 reviews</button>
# # <button aria-label="Save House on Parliament in your lists" data-value="Save" jsaction="pane.placeActions.save;keydown:pane.placeActions.save" jstcache="183" class="DVeyrd gm2-hairline-border section-action-chip-button" jsan="7.DVeyrd,7.gm2-hairline-border,7.section-action-chip-button,0.aria-label,0.data-value,0.jsaction"><img alt="Save" draggable="false" jstcache="184" src="//www.gstatic.com/images/icons/material/system_gm/2x/bookmark_border_gm_blue_18dp.png" class="EgL07d"><span jstcache="185" style="display:none"></span></button>
# # <button data-review-id="ChdDSUhNMG9nS0VJQ0FnSUNhNkx6QXRnRRAB" ved="1i:3,t:63707,e:2,p:NEEVYYyGMJuDtQbmipKoCA:57" aria-label=" See more " jstcache="641" jsaction="pane.review.expandReview" class="ODSEW-KoToPc-ShBeI gXqMYb-hSRGPd" jsan="7.ODSEW-KoToPc-ShBeI,7.gXqMYb-hSRGPd,0.data-review-id,0.ved,0.aria-label,22.jsaction">More</button>
# links

In [ ]:
# links[0].click()
# <selenium.webdriver.remote.webelement.WebElement (session="eec97e0683b6787c9aee03f79023ee0a", element="0e6c387d-433e-406e-87e1-3bf675b76b5c")>


In [ ]:
# scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc')

# driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
# # scrollable_div

In [ ]:
# # load all the reviews on the page
# def more_reviews(driver):
    
#     links = driver.find_elements_by_xpath('//*[contains(@aria-label,"More reviews")]') 
#     links = driver.find_elements_by_xpath('//*[contains(@aria-label,"More reviews")]/div/div[1]/div/div/div[45]/div/div/button/span') 
# #     //*[@id="pane"]/div/div[1]/div/div/div[45]/div/div/button/span


#     # has to change to match stucture
# #     print('LINKS HERE', links)
#     for l in links:
#         l.click()
#     time.sleep(5)       
# #     links = driver.find_elements_by_xpath('//button[@jsaction=\'pane.reviewChart.moreReviews\']')
# #     print('LINKS HERE', links)
# #     for l in links:
# #         l.click()
# #     time.sleep(2)


In [ ]:
# soup = BeautifulSoup(driver.page_source,'html.parser')
# buttons = soup.find_all('span', class_="wNNZR gm2-button-alt")
# buttons 
# # <div data-test="restaurant-cards"><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="58213" data-marketing="true" data-index="1" data-avt="eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ" data-promoted="true" data-marketing-ids="7" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/the-shore-club-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23690283.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23690283.jpg 105w, https://images.otstatic.com/prod/23690283/1/small.jpg 160w, https://images.otstatic.com/prod/23690283/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23690283.jpg" alt="A photo of The Shore Club - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/the-shore-club-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Shore Club - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Shore Club - Toronto</h6></a><span class="_2dqbEbHzyUbJA0-w3pFoig">Promoted</span></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/the-shore-club-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Shore Club - Toronto restaurant reviews (1546)">(1546)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Seafood • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 40 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=58213&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=1000&amp;pofids=&amp;hash=2059100605&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJjIjo3LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard&amp;i=1" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at The Shore Club - Toronto restaurant and redeem +1,000 pts">6:30 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">+1,000 pts</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=58213&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=1000&amp;pofids=&amp;hash=1380603130&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJjIjo3LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard&amp;i=1" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at The Shore Club - Toronto restaurant and redeem +1,000 pts">6:45 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">+1,000 pts</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=58213&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=1000&amp;pofids=&amp;hash=1877746809&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJjIjo3LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard&amp;i=1" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at The Shore Club - Toronto restaurant and redeem +1,000 pts">7:00 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">+1,000 pts</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=58213&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1465000001&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at The Shore Club - Toronto restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1068151" data-marketing="true" data-index="2" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/almond-butterfly-dundas-bistro-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/29419363.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/29419363.jpg 105w, https://images.otstatic.com/prod1/29419363/1/small.jpg 160w, https://images.otstatic.com/prod1/29419363/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/29419363.jpg" alt="A photo of Almond Butterfly Dundas | Bistro restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/almond-butterfly-dundas-bistro-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Almond Butterfly Dundas | Bistro restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Almond Butterfly Dundas | Bistro</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/almond-butterfly-dundas-bistro-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Almond Butterfly Dundas | Bistro restaurant reviews (98)">(98)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Little Portugal / Dundas West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 29 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1068151&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=801379371&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjc1OTI2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Almond Butterfly Dundas | Bistro restaurant" aria-describedby="tooltip-801379371">6:30 PM*<div id="tooltip-801379371" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1068151&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1577734803&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjc1OTI2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Almond Butterfly Dundas | Bistro restaurant" aria-describedby="tooltip-1577734803">6:45 PM*<div id="tooltip-1577734803" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1068151&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2908189481&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjc1OTI2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Almond Butterfly Dundas | Bistro restaurant" aria-describedby="tooltip-2908189481">7:00 PM*<div id="tooltip-2908189481" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1068151&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3658764952&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjc1OTI2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Almond Butterfly Dundas | Bistro restaurant" aria-describedby="tooltip-3658764952">7:15 PM*<div id="tooltip-3658764952" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1068151&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3320116689&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjc1OTI2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Almond Butterfly Dundas | Bistro restaurant" aria-describedby="tooltip-3320116689">7:30 PM*<div id="tooltip-3320116689" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="158806" data-marketing="true" data-index="3" data-avt="eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ" data-promoted="true" data-marketing-ids="1631911" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/bindia-indian-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/4/28231963.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/4/28231963.jpg 105w, https://images.otstatic.com/prod/28231963/4/small.jpg 160w, https://images.otstatic.com/prod/28231963/4/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/4/28231963.jpg" alt="A photo of Bindia Indian Bistro restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/bindia-indian-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Bindia Indian Bistro restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Bindia Indian Bistro</h6></a><span class="_2dqbEbHzyUbJA0-w3pFoig">Promoted</span></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/bindia-indian-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Bindia Indian Bistro restaurant reviews (207)">(207)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Indian • St. Lawrence Market</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 7 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=158806&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1833028184&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJjIjoxNjMxOTExLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bindia Indian Bistro restaurant" aria-describedby="tooltip-1833028184">4:45 PM*<div id="tooltip-1833028184" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=158806&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4041932765&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJjIjoxNjMxOTExLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bindia Indian Bistro restaurant" aria-describedby="tooltip-4041932765">5:00 PM*<div id="tooltip-4041932765" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1076746" data-marketing="true" data-index="4" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/buono-college-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/30938700.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/30938700.jpg 105w, https://images.otstatic.com/prod1/30938700/3/small.jpg 160w, https://images.otstatic.com/prod1/30938700/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/30938700.jpg" alt="A photo of Buono College restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/buono-college-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Buono College restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Buono College</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/buono-college-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Buono College restaurant reviews (110)">(110)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Little Italy / College St.</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 28 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1076746&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2421651529&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Buono College restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1076746&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1185927600&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Buono College restaurant">5:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="108508" data-marketing="true" data-index="5" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/chiado?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23667793.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23667793.jpg 105w, https://images.otstatic.com/prod/23667793/1/small.jpg 160w, https://images.otstatic.com/prod/23667793/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23667793.jpg" alt="A photo of Chiado restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/chiado?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Chiado restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Chiado</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.9 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.9</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/chiado?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Chiado restaurant reviews (968)">(968)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Portuguese • Little Italy / College St.</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 11 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=108508&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3500710096&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Chiado restaurant" aria-describedby="tooltip-3500710096">6:00 PM*<div id="tooltip-3500710096" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=108508&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2809416248&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Chiado restaurant" aria-describedby="tooltip-2809416248">6:15 PM*<div id="tooltip-2809416248" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=108508&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3166860187&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Chiado restaurant" aria-describedby="tooltip-3166860187">9:00 PM*<div id="tooltip-3166860187" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=108508&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3172239470&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Chiado restaurant" aria-describedby="tooltip-3172239470">9:15 PM*<div id="tooltip-3172239470" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1040248" data-marketing="true" data-index="6" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/founder-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/29367733.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/29367733.jpg 105w, https://images.otstatic.com/prod1/29367733/2/small.jpg 160w, https://images.otstatic.com/prod1/29367733/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/29367733.jpg" alt="A photo of Founder restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/founder-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Founder restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Founder</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/founder-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Founder restaurant reviews (35)">(35)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Global, International • Ossington</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 11 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1040248&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3905514624&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Founder restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1040248&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1112482005&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Founder restaurant">5:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1066510" data-marketing="true" data-index="7" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/tav-gnocchi-and-pizza-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/29083762.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/29083762.jpg 105w, https://images.otstatic.com/prod1/29083762/3/small.jpg 160w, https://images.otstatic.com/prod1/29083762/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/29083762.jpg" alt="A photo of Tav Gnocchi &amp; Pizza Bar restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/tav-gnocchi-and-pizza-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Tav Gnocchi &amp; Pizza Bar restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Tav Gnocchi &amp; Pizza Bar</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/tav-gnocchi-and-pizza-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Tav Gnocchi &amp; Pizza Bar restaurant reviews (98)">(98)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Little Italy / College St.</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 34 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1066510&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4250843431&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Tav Gnocchi &amp; Pizza Bar restaurant" aria-describedby="tooltip-4250843431">5:45 PM*<div id="tooltip-4250843431" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1066510&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=788545087&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Tav Gnocchi &amp; Pizza Bar restaurant" aria-describedby="tooltip-788545087">6:00 PM*<div id="tooltip-788545087" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1008718" data-marketing="true" data-index="8" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/old-school-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/29401612.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/29401612.jpg 105w, https://images.otstatic.com/prod1/29401612/1/small.png 160w, https://images.otstatic.com/prod1/29401612/1/medium.png 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/29401612.jpg" alt="A photo of Old School - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/old-school-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Old School - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Old School - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/old-school-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Old School - Toronto restaurant reviews (294)">(294)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Comfort Food • Ossington</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 61 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1008718&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2570428137&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Old School - Toronto restaurant" aria-describedby="tooltip-2570428137">6:30 PM*<div id="tooltip-2570428137" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1008718&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1816461799&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Old School - Toronto restaurant" aria-describedby="tooltip-1816461799">6:45 PM*<div id="tooltip-1816461799" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1008718&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3907228187&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Old School - Toronto restaurant" aria-describedby="tooltip-3907228187">7:00 PM*<div id="tooltip-3907228187" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1008718&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2852116994&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Old School - Toronto restaurant" aria-describedby="tooltip-2852116994">7:15 PM*<div id="tooltip-2852116994" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1008718&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1844736117&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Old School - Toronto restaurant" aria-describedby="tooltip-1844736117">7:30 PM*<div id="tooltip-1844736117" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1035061" data-marketing="true" data-index="9" data-avt="eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ" data-promoted="true" data-marketing-ids="1636276" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/chefs-hall-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/31006674.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/31006674.jpg 105w, https://images.otstatic.com/prod1/31006674/3/small.jpg 160w, https://images.otstatic.com/prod1/31006674/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/31006674.jpg" alt="A photo of Chefs Hall restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/chefs-hall-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Chefs Hall restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Chefs Hall</h6></a><span class="_2dqbEbHzyUbJA0-w3pFoig">Promoted</span></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/chefs-hall-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Chefs Hall restaurant reviews (122)">(122)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • American • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 25 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1035061&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1414186215&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJjIjoxNjM2Mjc2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at Chefs Hall restaurant" aria-describedby="tooltip-1414186215">5:45 PM*<div id="tooltip-1414186215" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1035061&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3556424038&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJjIjoxNjM2Mjc2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at Chefs Hall restaurant" aria-describedby="tooltip-3556424038">6:00 PM*<div id="tooltip-3556424038" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="22474" data-marketing="true" data-index="10" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/noce-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/24982287.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/24982287.jpg 105w, https://images.otstatic.com/prod/24982287/1/small.jpg 160w, https://images.otstatic.com/prod/24982287/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/24982287.jpg" alt="A photo of Noce Restaurant restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/noce-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Noce Restaurant restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Noce Restaurant</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/noce-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Noce Restaurant restaurant reviews (405)">(405)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 44 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=22474&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1721531151&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Noce Restaurant restaurant" aria-describedby="tooltip-1721531151">5:00 PM*<div id="tooltip-1721531151" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=22474&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3161331486&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Noce Restaurant restaurant" aria-describedby="tooltip-3161331486">5:15 PM*<div id="tooltip-3161331486" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1011109" data-marketing="true" data-index="11" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/fresh-on-crawford-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/31777682.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/31777682.jpg 105w, https://images.otstatic.com/prod1/31777682/3/small.jpg 160w, https://images.otstatic.com/prod1/31777682/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/31777682.jpg" alt="A photo of Fresh on Crawford restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/fresh-on-crawford-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Fresh on Crawford restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Fresh on Crawford</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/fresh-on-crawford-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Fresh on Crawford restaurant reviews (226)">(226)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Vegetarian / Vegan • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 37 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1011109&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1392568216&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Fresh on Crawford restaurant" aria-describedby="tooltip-1392568216">4:30 PM*<div id="tooltip-1392568216" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1011109&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4079167312&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Fresh on Crawford restaurant" aria-describedby="tooltip-4079167312">4:45 PM*<div id="tooltip-4079167312" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1011109&amp;d=2021-08-06T20%3A30%3A00&amp;sd=2021-08-06T20%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4016764583&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:30 PM Reserve table at Fresh on Crawford restaurant" aria-describedby="tooltip-4016764583">8:30 PM*<div id="tooltip-4016764583" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1011109&amp;d=2021-08-06T20%3A45%3A00&amp;sd=2021-08-06T20%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1971045958&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:45 PM Reserve table at Fresh on Crawford restaurant" aria-describedby="tooltip-1971045958">8:45 PM*<div id="tooltip-1971045958" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="995242" data-marketing="true" data-index="12" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/il-covo-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/26434838.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/26434838.jpg 105w, https://images.otstatic.com/prod/26434838/3/small.jpg 160w, https://images.otstatic.com/prod/26434838/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/26434838.jpg" alt="A photo of Il Covo restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/il-covo-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Il Covo restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Il Covo</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/il-covo-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Il Covo restaurant reviews (273)">(273)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Little Italy / College St.</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 23 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=995242&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=211868884&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Il Covo restaurant" aria-describedby="tooltip-211868884">5:00 PM*<div id="tooltip-211868884" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=995242&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2289051300&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Il Covo restaurant" aria-describedby="tooltip-2289051300">5:30 PM*<div id="tooltip-2289051300" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=995242&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2371679401&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Il Covo restaurant" aria-describedby="tooltip-2371679401">9:30 PM*<div id="tooltip-2371679401" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1006435" data-marketing="true" data-index="13" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/jules-bistro-queen-west-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/30218227.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/30218227.jpg 105w, https://images.otstatic.com/prod1/30218227/1/small.jpg 160w, https://images.otstatic.com/prod1/30218227/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/30218227.jpg" alt="A photo of Jules Bistro- Queen West restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/jules-bistro-queen-west-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Jules Bistro- Queen West restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Jules Bistro- Queen West</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/jules-bistro-queen-west-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Jules Bistro- Queen West restaurant reviews (332)">(332)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • French • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 48 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1006435&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2556950281&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Jules Bistro- Queen West restaurant" aria-describedby="tooltip-2556950281">6:30 PM*<div id="tooltip-2556950281" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1006435&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4210658173&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Jules Bistro- Queen West restaurant" aria-describedby="tooltip-4210658173">6:45 PM*<div id="tooltip-4210658173" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1006435&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1566017604&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Jules Bistro- Queen West restaurant" aria-describedby="tooltip-1566017604">7:00 PM*<div id="tooltip-1566017604" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1071769" data-marketing="true" data-index="14" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/the-commoner-bar-room-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/30991253.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/30991253.jpg 105w, https://images.otstatic.com/prod1/30991253/1/small.jpg 160w, https://images.otstatic.com/prod1/30991253/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/30991253.jpg" alt="A photo of The Commoner Bar Room restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/the-commoner-bar-room-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Commoner Bar Room restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Commoner Bar Room</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/the-commoner-bar-room-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Commoner Bar Room restaurant reviews (32)">(32)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary American • Little Italy / College St.</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 25 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1071769&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1150585052&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at The Commoner Bar Room restaurant" aria-describedby="tooltip-1150585052">6:30 PM*<div id="tooltip-1150585052" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1071769&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2430044774&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at The Commoner Bar Room restaurant" aria-describedby="tooltip-2430044774">6:45 PM*<div id="tooltip-2430044774" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1071769&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=915412333&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at The Commoner Bar Room restaurant" aria-describedby="tooltip-915412333">7:00 PM*<div id="tooltip-915412333" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1071769&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=422209161&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at The Commoner Bar Room restaurant" aria-describedby="tooltip-422209161">7:15 PM*<div id="tooltip-422209161" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1071769&amp;d=2021-08-06T19%3A45%3A00&amp;sd=2021-08-06T19%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3289940444&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:45 PM Reserve table at The Commoner Bar Room restaurant" aria-describedby="tooltip-3289940444">7:45 PM*<div id="tooltip-3289940444" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1194742" data-marketing="true" data-index="15" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/the-cottage-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/42304440.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/42304440.jpg 105w, https://images.otstatic.com/prod1/42304440/2/small.jpg 160w, https://images.otstatic.com/prod1/42304440/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/42304440.jpg" alt="A photo of The Cottage restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/the-cottage-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Newly added The Cottage restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Cottage</h6><span class="_1i427XmQ62ehaUl3OpWJrD">new</span></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/the-cottage-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Cottage restaurant reviews (23)">(23)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • American • Thornbury</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 18 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1194742&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2603225107&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at The Cottage restaurant" aria-describedby="tooltip-2603225107">6:00 PM*<div id="tooltip-2603225107" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1194742&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=21655487&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:15 PM Reserve table at The Cottage restaurant">6:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1194742&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1259598229&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at The Cottage restaurant" aria-describedby="tooltip-1259598229">9:00 PM*<div id="tooltip-1259598229" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1194742&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=448716019&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:15 PM Reserve table at The Cottage restaurant" aria-describedby="tooltip-448716019">9:15 PM*<div id="tooltip-448716019" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="270817" data-marketing="true" data-index="16" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-marketing-ids="7" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/stk-toronto-1?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42515234.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42515234.jpg 105w, https://images.otstatic.com/prod1/42515234/3/small.jpg 160w, https://images.otstatic.com/prod1/42515234/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42515234.jpg" alt="A photo of STK - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/stk-toronto-1?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="STK - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">STK - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.2 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.2</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/stk-toronto-1?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview STK - Toronto restaurant reviews (1381)">(1381)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 172 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=270817&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=1000&amp;pofids=&amp;hash=1398298665&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo3LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard&amp;i=1" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at STK - Toronto restaurant and redeem +1,000 pts" aria-describedby="tooltip-1398298665">5:15 PM*<div id="tooltip-1398298665" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">+1,000 pts</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=270817&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=1000&amp;pofids=&amp;hash=3536786321&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo3LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard&amp;i=1" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at STK - Toronto restaurant and redeem +1,000 pts" aria-describedby="tooltip-3536786321">5:30 PM*<div id="tooltip-3536786321" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">+1,000 pts</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="334561" data-marketing="true" data-index="17" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/liberty-commons-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/29382422.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/29382422.jpg 105w, https://images.otstatic.com/prod1/29382422/2/small.jpg 160w, https://images.otstatic.com/prod1/29382422/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/29382422.jpg" alt="A photo of Liberty Commons restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/liberty-commons-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Liberty Commons restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Liberty Commons</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/liberty-commons-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Liberty Commons restaurant reviews (447)">(447)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Comfort Food • Liberty Village</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 42 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=334561&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1980836386&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:30 PM Reserve table at Liberty Commons restaurant" aria-describedby="tooltip-1980836386">9:30 PM*<div id="tooltip-1980836386" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1027885" data-marketing="true" data-index="18" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/lapinou-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/28464134.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/28464134.jpg 105w, https://images.otstatic.com/prod/28464134/3/small.jpg 160w, https://images.otstatic.com/prod/28464134/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/28464134.jpg" alt="A photo of Lapinou restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/lapinou-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Lapinou restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Lapinou</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/lapinou-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Lapinou restaurant reviews (323)">(323)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary French • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 102 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1027885&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=694705800&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Lapinou restaurant">5:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1027885&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3860730660&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Lapinou restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1026244" data-marketing="true" data-index="19" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/ristorante-rosina-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25797149.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25797149.jpg 105w, https://images.otstatic.com/prod/25797149/1/small.jpg 160w, https://images.otstatic.com/prod/25797149/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25797149.jpg" alt="A photo of Ristorante Rosina restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/ristorante-rosina-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Ristorante Rosina restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Ristorante Rosina</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/ristorante-rosina-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Ristorante Rosina restaurant reviews (266)">(266)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 41 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1026244&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3645943747&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Ristorante Rosina restaurant" aria-describedby="tooltip-3645943747">4:30 PM*<div id="tooltip-3645943747" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1070242" data-marketing="true" data-index="20" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/come-see-me-toranto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/29436700.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/29436700.jpg 105w, https://images.otstatic.com/prod1/29436700/2/small.jpg 160w, https://images.otstatic.com/prod1/29436700/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/29436700.jpg" alt="A photo of Come See Me restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/come-see-me-toranto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Come See Me restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Come See Me</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.2 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.2</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/come-see-me-toranto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Come See Me restaurant reviews (12)">(12)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Cocktail Bar • Little Italy / College St.</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 7 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1070242&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4051038136&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Come See Me restaurant" aria-describedby="tooltip-4051038136">6:15 PM*<div id="tooltip-4051038136" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1070242&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1956965024&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Come See Me restaurant" aria-describedby="tooltip-1956965024">6:30 PM*<div id="tooltip-1956965024" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="24703" data-marketing="true" data-index="21" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/drake-hotel?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/30885685.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/30885685.jpg 105w, https://images.otstatic.com/prod1/30885685/3/small.png 160w, https://images.otstatic.com/prod1/30885685/3/medium.png 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/30885685.jpg" alt="A photo of Drake Hotel restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/drake-hotel?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Drake Hotel restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Drake Hotel</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/drake-hotel?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Drake Hotel restaurant reviews (807)">(807)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 112 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=24703&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1662744123&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Drake Hotel restaurant" aria-describedby="tooltip-1662744123">5:30 PM*<div id="tooltip-1662744123" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=24703&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2861306112&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at Drake Hotel restaurant" aria-describedby="tooltip-2861306112">5:45 PM*<div id="tooltip-2861306112" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1045186" data-marketing="true" data-index="22" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/ascari-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/28028163.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/28028163.jpg 105w, https://images.otstatic.com/prod/28028163/3/small.jpg 160w, https://images.otstatic.com/prod/28028163/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/28028163.jpg" alt="A photo of Ascari King Street restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/ascari-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Ascari King Street restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Ascari King Street</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/ascari-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Ascari King Street restaurant reviews (272)">(272)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 48 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1045186&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3507838098&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Ascari King Street restaurant" aria-describedby="tooltip-3507838098">5:00 PM*<div id="tooltip-3507838098" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1045186&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4147472074&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at Ascari King Street restaurant" aria-describedby="tooltip-4147472074">9:00 PM*<div id="tooltip-4147472074" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1045186&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=562684550&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:15 PM Reserve table at Ascari King Street restaurant" aria-describedby="tooltip-562684550">9:15 PM*<div id="tooltip-562684550" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="242605" data-marketing="true" data-index="23" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/caffino-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/24942192.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/24942192.jpg 105w, https://images.otstatic.com/prod/24942192/1/small.jpg 160w, https://images.otstatic.com/prod/24942192/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/24942192.jpg" alt="A photo of Caffino restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/caffino-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Caffino restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Caffino</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/caffino-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Caffino restaurant reviews (311)">(311)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Liberty Village</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 28 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=242605&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2799614838&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Caffino restaurant" aria-describedby="tooltip-2799614838">5:45 PM*<div id="tooltip-2799614838" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=242605&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2504938860&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Caffino restaurant" aria-describedby="tooltip-2504938860">6:00 PM*<div id="tooltip-2504938860" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=242605&amp;d=2021-08-06T20%3A15%3A00&amp;sd=2021-08-06T20%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3219097965&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Caffino restaurant" aria-describedby="tooltip-3219097965">8:15 PM*<div id="tooltip-3219097965" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=242605&amp;d=2021-08-06T20%3A30%3A00&amp;sd=2021-08-06T20%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4002472129&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Caffino restaurant" aria-describedby="tooltip-4002472129">8:30 PM*<div id="tooltip-4002472129" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="105955" data-marketing="true" data-index="24" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/cibo-wine-bar-king-st-west?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23667005.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23667005.jpg 105w, https://images.otstatic.com/prod/23667005/1/small.jpg 160w, https://images.otstatic.com/prod/23667005/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23667005.jpg" alt="A photo of Cibo Wine Bar King St West restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/cibo-wine-bar-king-st-west?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Cibo Wine Bar King St West restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Cibo Wine Bar King St West</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/cibo-wine-bar-king-st-west?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Cibo Wine Bar King St West restaurant reviews (2348)">(2348)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 77 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=105955&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4159641185&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Cibo Wine Bar King St West restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=105955&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2469132690&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Cibo Wine Bar King St West restaurant">5:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1042684" data-marketing="true" data-index="25" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/shook-kitchen-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/4/26529378.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/4/26529378.jpg 105w, https://images.otstatic.com/prod/26529378/4/small.jpg 160w, https://images.otstatic.com/prod/26529378/4/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/4/26529378.jpg" alt="A photo of Shook Kitchen restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/shook-kitchen-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Shook Kitchen restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Shook Kitchen</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/shook-kitchen-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Shook Kitchen restaurant reviews (183)">(183)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Middle Eastern • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 73 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1042684&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2181706803&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Shook Kitchen restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1042684&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2292883360&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Shook Kitchen restaurant">5:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1037560" data-marketing="true" data-index="26" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/stelvio-dundas-west-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/42467739.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/42467739.jpg 105w, https://images.otstatic.com/prod1/42467739/2/small.png 160w, https://images.otstatic.com/prod1/42467739/2/medium.png 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/42467739.jpg" alt="A photo of Stelvio restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/stelvio-dundas-west-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Stelvio restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Stelvio</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/stelvio-dundas-west-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Stelvio restaurant reviews (114)">(114)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Little Portugal / Dundas West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 21 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1037560&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=352567167&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Stelvio restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1037560&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1720989238&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Stelvio restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1037560&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=114376091&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Stelvio restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1037560&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1553252292&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Stelvio restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1037560&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3691644366&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Stelvio restaurant">7:30 PM</a></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1043089" data-marketing="true" data-index="27" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/xango-toranto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42302063.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42302063.jpg 105w, https://images.otstatic.com/prod1/42302063/3/small.jpg 160w, https://images.otstatic.com/prod1/42302063/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42302063.jpg" alt="A photo of Xango restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/xango-toranto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Xango restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Xango</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.1 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.1</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Excellent</span><a href="https://www.opentable.com/r/xango-toranto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Xango restaurant reviews (253)">(253)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Asian • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 181 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1043089&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2175495661&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Xango restaurant">5:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1043089&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1489303495&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at Xango restaurant">5:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="486547" data-marketing="true" data-index="28" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/peter-pan-bistro-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25557100.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25557100.jpg 105w, https://images.otstatic.com/prod/25557100/1/small.jpg 160w, https://images.otstatic.com/prod/25557100/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25557100.jpg" alt="A photo of Peter Pan Bistro restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/peter-pan-bistro-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Peter Pan Bistro restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Peter Pan Bistro</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/peter-pan-bistro-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Peter Pan Bistro restaurant reviews (206)">(206)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Wine Bar • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 34 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=486547&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4149874657&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Peter Pan Bistro restaurant" aria-describedby="tooltip-4149874657">5:00 PM*<div id="tooltip-4149874657" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=486547&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1153073318&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=highTop&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Peter Pan Bistro restaurant" aria-describedby="tooltip-1153073318">5:15 PM*<div id="tooltip-1153073318" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1014526" data-marketing="true" data-index="29" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/cano-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/41816588.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/41816588.jpg 105w, https://images.otstatic.com/prod1/41816588/1/small.jpg 160w, https://images.otstatic.com/prod1/41816588/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/41816588.jpg" alt="A photo of Cano Restaurant restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/cano-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Cano Restaurant restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Cano Restaurant</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/cano-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Cano Restaurant restaurant reviews (482)">(482)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • St. Clair West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 82 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1014526&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4159758157&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo1NzUwNDEsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Cano Restaurant restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1014526&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2814601570&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo1NzUwNDEsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Cano Restaurant restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1018396" data-marketing="true" data-index="30" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/alma-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/29626303.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/29626303.jpg 105w, https://images.otstatic.com/prod1/29626303/1/small.jpg 160w, https://images.otstatic.com/prod1/29626303/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/29626303.jpg" alt="A photo of Alma restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/alma-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Alma restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Alma</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/alma-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Alma restaurant reviews (128)">(128)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary Asian • Koreatown</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 36 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1018396&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1204001020&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Alma restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1018396&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3242613538&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Alma restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="26239" data-marketing="true" data-index="31" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/blu-ristorante?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/31721970.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/31721970.jpg 105w, https://images.otstatic.com/prod1/31721970/3/small.png 160w, https://images.otstatic.com/prod1/31721970/3/medium.png 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/31721970.jpg" alt="A photo of Blu Ristorante restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/blu-ristorante?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Blu Ristorante restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Blu Ristorante</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/blu-ristorante?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Blu Ristorante restaurant reviews (1990)">(1990)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 109 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=26239&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=672671782&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Blu Ristorante restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=26239&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3107315886&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Blu Ristorante restaurant">5:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1082035" data-marketing="true" data-index="32" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/minami-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/32402198.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/32402198.jpg 105w, https://images.otstatic.com/prod1/32402198/2/small.png 160w, https://images.otstatic.com/prod1/32402198/2/medium.png 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/32402198.jpg" alt="A photo of Minami Restaurant - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/minami-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Minami Restaurant - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Minami Restaurant - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/minami-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Minami Restaurant - Toronto restaurant reviews (68)">(68)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Japanese • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 64 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1082035&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=728063163&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Minami Restaurant - Toronto restaurant" aria-describedby="tooltip-728063163">5:15 PM*<div id="tooltip-728063163" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1082035&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2632658443&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Minami Restaurant - Toronto restaurant" aria-describedby="tooltip-2632658443">5:30 PM*<div id="tooltip-2632658443" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1082035&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1147873650&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:30 PM Reserve table at Minami Restaurant - Toronto restaurant" aria-describedby="tooltip-1147873650">9:30 PM*<div id="tooltip-1147873650" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="112777" data-marketing="true" data-index="33" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/byblos-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23669081.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23669081.jpg 105w, https://images.otstatic.com/prod/23669081/1/small.jpg 160w, https://images.otstatic.com/prod/23669081/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23669081.jpg" alt="A photo of Byblos - Downtown restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/byblos-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Byblos - Downtown restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Byblos - Downtown</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/byblos-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Byblos - Downtown restaurant reviews (2348)">(2348)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Mediterranean • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 73 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=112777&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2815767185&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:15 PM Reserve table at Byblos - Downtown restaurant">9:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=112777&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3978330297&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:30 PM Reserve table at Byblos - Downtown restaurant">9:30 PM</a></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="3636" data-marketing="true" data-index="34" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/bymark?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25139499.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25139499.jpg 105w, https://images.otstatic.com/prod/25139499/1/small.jpg 160w, https://images.otstatic.com/prod/25139499/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25139499.jpg" alt="A photo of Bymark restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/bymark?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Bymark restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Bymark</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/bymark?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Bymark restaurant reviews (1484)">(1484)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Continental • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 82 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=3636&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1859569725&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bymark restaurant" aria-describedby="tooltip-1859569725">5:30 PM*<div id="tooltip-1859569725" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1050574" data-marketing="true" data-index="35" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/ii-patio-di-eataly-with-aperol-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/42231906.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/42231906.jpg 105w, https://images.otstatic.com/prod1/42231906/2/small.jpg 160w, https://images.otstatic.com/prod1/42231906/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/42231906.jpg" alt="A photo of Il Patio di Eataly with Aperol restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/ii-patio-di-eataly-with-aperol-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Il Patio di Eataly with Aperol restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Il Patio di Eataly with Aperol</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/ii-patio-di-eataly-with-aperol-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Il Patio di Eataly with Aperol restaurant reviews (322)">(322)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 59 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1050574&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=930274128&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Il Patio di Eataly with Aperol restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="995431" data-marketing="true" data-index="36" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/fresh-on-bloor-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/32049474.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/32049474.jpg 105w, https://images.otstatic.com/prod1/32049474/2/small.jpg 160w, https://images.otstatic.com/prod1/32049474/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/32049474.jpg" alt="A photo of Fresh on Bloor restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/fresh-on-bloor-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Fresh on Bloor restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Fresh on Bloor</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.9 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.9</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/fresh-on-bloor-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Fresh on Bloor restaurant reviews (217)">(217)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Vegetarian / Vegan • The Annex</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 14 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=995431&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3713050512&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Fresh on Bloor restaurant" aria-describedby="tooltip-3713050512">5:15 PM*<div id="tooltip-3713050512" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=995431&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2066821576&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Fresh on Bloor restaurant" aria-describedby="tooltip-2066821576">5:30 PM*<div id="tooltip-2066821576" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=995431&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1691417788&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at Fresh on Bloor restaurant" aria-describedby="tooltip-1691417788">9:00 PM*<div id="tooltip-1691417788" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="73816" data-marketing="true" data-index="37" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/terroni-adelaide-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42309438.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42309438.jpg 105w, https://images.otstatic.com/prod1/42309438/3/small.jpg 160w, https://images.otstatic.com/prod1/42309438/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42309438.jpg" alt="A photo of Terroni Adelaide restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/terroni-adelaide-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Terroni Adelaide restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Terroni Adelaide</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/terroni-adelaide-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Terroni Adelaide restaurant reviews (3891)">(3891)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 161 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=73816&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=170220574&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Terroni Adelaide restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=73816&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1907115299&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Terroni Adelaide restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1011106" data-marketing="true" data-index="38" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/fresh-on-spadina-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/26172616.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/26172616.jpg 105w, https://images.otstatic.com/prod/26172616/2/small.jpg 160w, https://images.otstatic.com/prod/26172616/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/26172616.jpg" alt="A photo of Fresh on Spadina restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/fresh-on-spadina-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Fresh on Spadina restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Fresh on Spadina</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/fresh-on-spadina-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Fresh on Spadina restaurant reviews (233)">(233)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Vegetarian / Vegan • Chinatown / Kensington Market</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 88 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1011106&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4066829162&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:15 PM Reserve table at Fresh on Spadina restaurant">6:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1011106&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1247856275&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Fresh on Spadina restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1011106&amp;d=2021-08-06T20%3A15%3A00&amp;sd=2021-08-06T20%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2658104395&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Fresh on Spadina restaurant" aria-describedby="tooltip-2658104395">8:15 PM*<div id="tooltip-2658104395" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1011106&amp;d=2021-08-06T20%3A30%3A00&amp;sd=2021-08-06T20%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3285122953&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Fresh on Spadina restaurant" aria-describedby="tooltip-3285122953">8:30 PM*<div id="tooltip-3285122953" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="73561" data-marketing="true" data-index="39" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/tutti-matti?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23695518.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23695518.jpg 105w, https://images.otstatic.com/prod/23695518/1/small.jpg 160w, https://images.otstatic.com/prod/23695518/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23695518.jpg" alt="A photo of Tutti Matti restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/tutti-matti?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Tutti Matti restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Tutti Matti</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/tutti-matti?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Tutti Matti restaurant reviews (918)">(918)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Entertainment District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 14 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=73561&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=486784005&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Tutti Matti restaurant" aria-describedby="tooltip-486784005">6:30 PM*<div id="tooltip-486784005" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=73561&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=705187073&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Tutti Matti restaurant" aria-describedby="tooltip-705187073">6:45 PM*<div id="tooltip-705187073" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=73561&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1520960647&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Tutti Matti restaurant" aria-describedby="tooltip-1520960647">7:00 PM*<div id="tooltip-1520960647" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=73561&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2711577433&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Tutti Matti restaurant" aria-describedby="tooltip-2711577433">7:15 PM*<div id="tooltip-2711577433" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=73561&amp;d=2021-08-06T19%3A45%3A00&amp;sd=2021-08-06T19%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=290254635&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:45 PM Reserve table at Tutti Matti restaurant" aria-describedby="tooltip-290254635">7:45 PM*<div id="tooltip-290254635" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="32068" data-marketing="true" data-index="40" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/hot-house-restaurant-and-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/30885690.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/30885690.jpg 105w, https://images.otstatic.com/prod1/30885690/3/small.jpg 160w, https://images.otstatic.com/prod1/30885690/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/30885690.jpg" alt="A photo of HOTHOUSE restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/hot-house-restaurant-and-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="HOTHOUSE restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">HOTHOUSE</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/hot-house-restaurant-and-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview HOTHOUSE restaurant reviews (1465)">(1465)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • International • St. Lawrence Market</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 113 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=32068&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3080786028&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at HOTHOUSE restaurant" aria-describedby="tooltip-3080786028">4:30 PM*<div id="tooltip-3080786028" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=32068&amp;d=2021-08-06T20%3A45%3A00&amp;sd=2021-08-06T20%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2357384091&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:45 PM Reserve table at HOTHOUSE restaurant" aria-describedby="tooltip-2357384091">8:45 PM*<div id="tooltip-2357384091" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=32068&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=364337858&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at HOTHOUSE restaurant" aria-describedby="tooltip-364337858">9:00 PM*<div id="tooltip-364337858" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="189151" data-marketing="true" data-index="41" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/the-keg-steakhouse-and-bar-esplanade?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/30103605.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/30103605.jpg 105w, https://images.otstatic.com/prod1/30103605/1/small.jpg 160w, https://images.otstatic.com/prod1/30103605/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/30103605.jpg" alt="A photo of The Keg Steakhouse + Bar - Esplanade restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/the-keg-steakhouse-and-bar-esplanade?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Keg Steakhouse + Bar - Esplanade restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Keg Steakhouse + Bar - Esplanade</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/the-keg-steakhouse-and-bar-esplanade?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Keg Steakhouse + Bar - Esplanade restaurant reviews (842)">(842)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 104 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=189151&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=986630658&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at The Keg Steakhouse + Bar - Esplanade restaurant" aria-describedby="tooltip-986630658">4:30 PM*<div id="tooltip-986630658" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=189151&amp;d=2021-08-06T20%3A45%3A00&amp;sd=2021-08-06T20%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3588577275&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=bar&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at The Keg Steakhouse + Bar - Esplanade restaurant" aria-describedby="tooltip-3588577275">8:45 PM*<div id="tooltip-3588577275" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Bar</li><li>High Top</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=189151&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1564962759&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=bar&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at The Keg Steakhouse + Bar - Esplanade restaurant" aria-describedby="tooltip-1564962759">9:00 PM*<div id="tooltip-1564962759" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Bar</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1051627" data-marketing="true" data-index="42" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/lucia-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/28605720.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/28605720.jpg 105w, https://images.otstatic.com/prod/28605720/3/small.jpg 160w, https://images.otstatic.com/prod/28605720/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/28605720.jpg" alt="A photo of Lucia - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/lucia-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Lucia - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Lucia - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/lucia-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Lucia - Toronto restaurant reviews (206)">(206)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • The Junction</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 36 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1051627&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1240574529&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Lucia - Toronto restaurant" aria-describedby="tooltip-1240574529">5:30 PM*<div id="tooltip-1240574529" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1051627&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1120104164&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at Lucia - Toronto restaurant" aria-describedby="tooltip-1120104164">6:00 PM*<div id="tooltip-1120104164" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="151372" data-marketing="true" data-index="43" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/pizzeria-libretto-university?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23674567.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23674567.jpg 105w, https://images.otstatic.com/prod/23674567/1/small.jpg 160w, https://images.otstatic.com/prod/23674567/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23674567.jpg" alt="A photo of Pizzeria Libretto University restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/pizzeria-libretto-university?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Pizzeria Libretto University restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Pizzeria Libretto University</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/pizzeria-libretto-university?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Pizzeria Libretto University restaurant reviews (565)">(565)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Pizzeria • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 24 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=151372&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2833793339&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Pizzeria Libretto University restaurant" aria-describedby="tooltip-2833793339">4:30 PM*<div id="tooltip-2833793339" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=151372&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=345521892&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Pizzeria Libretto University restaurant" aria-describedby="tooltip-345521892">4:45 PM*<div id="tooltip-345521892" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="254872" data-marketing="true" data-index="44" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/buono-queen-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/42466970.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/42466970.jpg 105w, https://images.otstatic.com/prod1/42466970/2/small.jpg 160w, https://images.otstatic.com/prod1/42466970/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/42466970.jpg" alt="A photo of Buono Queen restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/buono-queen-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Buono Queen restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Buono Queen</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/buono-queen-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Buono Queen restaurant reviews (149)">(149)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • King West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 25 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=254872&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1376735698&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at Buono Queen restaurant">5:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=254872&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1888570723&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at Buono Queen restaurant">6:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1041283" data-marketing="true" data-index="45" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/piri-piri-grill-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/26390662.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/26390662.jpg 105w, https://images.otstatic.com/prod/26390662/3/small.jpg 160w, https://images.otstatic.com/prod/26390662/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/26390662.jpg" alt="A photo of Piri Piri Grill restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/piri-piri-grill-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Piri Piri Grill restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Piri Piri Grill</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/piri-piri-grill-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Piri Piri Grill restaurant reviews (124)">(124)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Mediterranean • The Junction</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 22 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1041283&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4205118562&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Piri Piri Grill restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1041283&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2879066947&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Piri Piri Grill restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1041283&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1087933392&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Piri Piri Grill restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1041283&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=175264139&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Piri Piri Grill restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="4671" data-marketing="true" data-index="46" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/ki-modern-japanese-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25646381.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25646381.jpg 105w, https://images.otstatic.com/prod/25646381/1/small.jpg 160w, https://images.otstatic.com/prod/25646381/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25646381.jpg" alt="A photo of Ki Modern Japanese + Bar - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/ki-modern-japanese-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Ki Modern Japanese + Bar - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Ki Modern Japanese + Bar - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/ki-modern-japanese-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Ki Modern Japanese + Bar - Toronto restaurant reviews (1821)">(1821)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Japanese • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 68 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=4671&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2753179783&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Ki Modern Japanese + Bar - Toronto restaurant" aria-describedby="tooltip-2753179783">5:00 PM*<div id="tooltip-2753179783" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=4671&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2899154265&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Ki Modern Japanese + Bar - Toronto restaurant" aria-describedby="tooltip-2899154265">5:15 PM*<div id="tooltip-2899154265" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=4671&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1120783538&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Ki Modern Japanese + Bar - Toronto restaurant" aria-describedby="tooltip-1120783538">9:30 PM*<div id="tooltip-1120783538" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="160681" data-marketing="true" data-index="47" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r-and-d-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23675999.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23675999.jpg 105w, https://images.otstatic.com/prod/23675999/1/small.jpg 160w, https://images.otstatic.com/prod/23675999/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23675999.jpg" alt="A photo of R&amp;D Restaurant - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r-and-d-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="R&amp;D Restaurant - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">R&amp;D Restaurant - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r-and-d-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview R&amp;D Restaurant - Toronto restaurant reviews (860)">(860)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary Asian • Chinatown / Kensington Market</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 17 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=160681&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3650474673&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at R&amp;D Restaurant - Toronto restaurant">5:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=160681&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3310478497&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at R&amp;D Restaurant - Toronto restaurant">5:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1069126" data-marketing="true" data-index="48" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/scarlett-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/29560975.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/29560975.jpg 105w, https://images.otstatic.com/prod1/29560975/3/small.jpg 160w, https://images.otstatic.com/prod1/29560975/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/29560975.jpg" alt="A photo of Scarlett restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/scarlett-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Scarlett restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Scarlett</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="3.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">3.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Very Good</span><a href="https://www.opentable.com/r/scarlett-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Scarlett restaurant reviews (17)">(17)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • American • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 23 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1069126&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1577448687&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at Scarlett restaurant" aria-describedby="tooltip-1577448687">6:00 PM*<div id="tooltip-1577448687" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1069126&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=352081957&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:15 PM Reserve table at Scarlett restaurant" aria-describedby="tooltip-352081957">6:15 PM*<div id="tooltip-352081957" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="247195" data-marketing="true" data-index="49" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/bampot-house-of-tea-and-board-games-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/32433448.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/32433448.jpg 105w, https://images.otstatic.com/prod1/32433448/1/small.jpg 160w, https://images.otstatic.com/prod1/32433448/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/32433448.jpg" alt="A photo of Bampot House of Tea &amp; Board Games restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/bampot-house-of-tea-and-board-games-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Bampot House of Tea &amp; Board Games restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Bampot House of Tea &amp; Board Games</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/bampot-house-of-tea-and-board-games-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Bampot House of Tea &amp; Board Games restaurant reviews (8)">(8)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • International • Harbord Village</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 1 time today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=247195&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4126244482&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=35402%2C35022%2C35021%2C34007%2C34006%2C33923%2C33921%2C36811&amp;ra=&amp;isMandatory=true&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bampot House of Tea &amp; Board Games restaurant and redeem Experiences" aria-describedby="tooltip-4126244482">6:30 PM*<div id="tooltip-4126244482" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=247195&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2927257829&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=35402%2C35022%2C35021%2C34007%2C34006%2C33923%2C33921%2C36811&amp;ra=&amp;isMandatory=true&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bampot House of Tea &amp; Board Games restaurant and redeem Experiences" aria-describedby="tooltip-2927257829">6:45 PM*<div id="tooltip-2927257829" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=247195&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=472270457&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=35402%2C35022%2C35021%2C34007%2C34006%2C33923%2C33921%2C36811&amp;ra=&amp;isMandatory=true&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bampot House of Tea &amp; Board Games restaurant and redeem Experiences" aria-describedby="tooltip-472270457">7:00 PM*<div id="tooltip-472270457" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=247195&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=690678646&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=35402%2C35022%2C35021%2C34007%2C34006%2C33923%2C33921%2C36811&amp;ra=&amp;isMandatory=true&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bampot House of Tea &amp; Board Games restaurant and redeem Experiences" aria-describedby="tooltip-690678646">7:15 PM*<div id="tooltip-690678646" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=247195&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3107453459&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=35402%2C35022%2C35021%2C34007%2C34006%2C33923%2C33921%2C36811&amp;ra=&amp;isMandatory=true&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Bampot House of Tea &amp; Board Games restaurant and redeem Experiences" aria-describedby="tooltip-3107453459">7:30 PM*<div id="tooltip-3107453459" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li></ul><div class="LDZNZTbXAbWJPLySat9S8"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSpecials"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M13.5,5.5 L13.5,5 C13.5,4.44771525 13.9477153,4 14.5,4 C15.0522847,4 15.5,4.44771525 15.5,5 L15.5,5.5 C15.5,7.709139 17.290861,9.5 19.5,9.5 L20,9.5 C20.5522847,9.5 21,9.94771525 21,10.5 C21,11.0522847 20.5522847,11.5 20,11.5 L19.5,11.5 C17.290861,11.5 15.5,13.290861 15.5,15.5 L15.5,16 C15.5,16.5522847 15.0522847,17 14.5,17 C13.9477153,17 13.5,16.5522847 13.5,16 L13.5,15.5 C13.5,13.290861 11.709139,11.5 9.5,11.5 L9,11.5 C8.44771525,11.5 8,11.0522847 8,10.5 C8,9.94771525 8.44771525,9.5 9,9.5 L9.5,9.5 C11.709139,9.5 13.5,7.709139 13.5,5.5 Z M4.5,12 C3.67157288,12 3,11.3284271 3,10.5 C3,9.67157288 3.67157288,9 4.5,9 C5.32842712,9 6,9.67157288 6,10.5 C6,11.3284271 5.32842712,12 4.5,12 Z M9,16 L10,16 C10.5522847,16 11,16.4477153 11,17 C11,17.5522847 10.5522847,18 10,18 L9,18 L9,19 C9,19.5522847 8.55228475,20 8,20 C7.44771525,20 7,19.5522847 7,19 L7,18 L6,18 C5.44771525,18 5,17.5522847 5,17 C5,16.4477153 5.44771525,16 6,16 L7,16 L7,15 C7,14.4477153 7.44771525,14 8,14 C8.55228475,14 9,14.4477153 9,15 L9,16 Z M12.8177951,10.5 C13.4841844,10.9430742 14.0569258,11.5158156 14.5,12.1822049 C14.9430742,11.5158156 15.5158156,10.9430742 16.1822049,10.5 C15.5158156,10.0569258 14.9430742,9.48418437 14.5,8.81779512 C14.0569258,9.48418437 13.4841844,10.0569258 12.8177951,10.5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Top experience</span></div><ul id="special-offers-list" class="_3xeXFm_LD6SL449SwaQeIk"><li class="LG3S6X7iyYg5Ejlp-bCZz">Bampot Bohemian High Tea + Board Games</li></ul></div><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1195462" data-marketing="true" data-index="50" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/tacotaco-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/42378228.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/42378228.jpg 105w, https://images.otstatic.com/prod1/42378228/2/small.jpg 160w, https://images.otstatic.com/prod1/42378228/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/42378228.jpg" alt="A photo of TacoTaco restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/tacotaco-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Newly added TacoTaco restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">TacoTaco</h6><span class="_1i427XmQ62ehaUl3OpWJrD">new</span></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/tacotaco-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview TacoTaco restaurant reviews (4)">(4)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Chinatown / Kensington Market</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 11 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1195462&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2436943953&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjMyMTkxLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at TacoTaco restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1195462&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1731815062&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjMyMTkxLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at TacoTaco restaurant">5:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="253213" data-marketing="true" data-index="51" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/soco-kitchen-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/28118827.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/28118827.jpg 105w, https://images.otstatic.com/prod/28118827/2/small.jpg 160w, https://images.otstatic.com/prod/28118827/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/28118827.jpg" alt="A photo of Soco Kitchen + Bar restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/soco-kitchen-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Soco Kitchen + Bar restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Soco Kitchen + Bar</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.1 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.1</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Excellent</span><a href="https://www.opentable.com/r/soco-kitchen-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Soco Kitchen + Bar restaurant reviews (259)">(259)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • International • Harbourfront</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 126 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=253213&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=620083010&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Soco Kitchen + Bar restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=253213&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3043552907&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Soco Kitchen + Bar restaurant" aria-describedby="tooltip-3043552907">6:30 PM*<div id="tooltip-3043552907" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="215137" data-marketing="true" data-index="52" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/bar-reyna-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25024519.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25024519.jpg 105w, https://images.otstatic.com/prod/25024519/1/small.jpg 160w, https://images.otstatic.com/prod/25024519/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25024519.jpg" alt="A photo of Bar Reyna restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/bar-reyna-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Bar Reyna restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Bar Reyna</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/bar-reyna-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Bar Reyna restaurant reviews (636)">(636)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Mediterranean • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 38 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=215137&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3362307097&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Bar Reyna restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="145777" data-marketing="true" data-index="53" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/lobby-lounge-at-shangri-la-hotel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/31293525.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/31293525.jpg 105w, https://images.otstatic.com/prod1/31293525/3/small.jpg 160w, https://images.otstatic.com/prod1/31293525/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/31293525.jpg" alt="A photo of Lobby Lounge at Shangri-La Hotel, Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/lobby-lounge-at-shangri-la-hotel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Lobby Lounge at Shangri-La Hotel, Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Lobby Lounge at Shangri-La Hotel, Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/lobby-lounge-at-shangri-la-hotel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Lobby Lounge at Shangri-La Hotel, Toronto restaurant reviews (114)">(114)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 46 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=145777&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=307886297&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=35101&amp;ra=&amp;isMandatory=true&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:15 PM Reserve table at Lobby Lounge at Shangri-La Hotel, Toronto restaurant and redeem Experiences">6:15 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=145777&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=607507649&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=35101&amp;ra=&amp;isMandatory=true&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Lobby Lounge at Shangri-La Hotel, Toronto restaurant and redeem Experiences">6:30 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="LDZNZTbXAbWJPLySat9S8"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSpecials"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M13.5,5.5 L13.5,5 C13.5,4.44771525 13.9477153,4 14.5,4 C15.0522847,4 15.5,4.44771525 15.5,5 L15.5,5.5 C15.5,7.709139 17.290861,9.5 19.5,9.5 L20,9.5 C20.5522847,9.5 21,9.94771525 21,10.5 C21,11.0522847 20.5522847,11.5 20,11.5 L19.5,11.5 C17.290861,11.5 15.5,13.290861 15.5,15.5 L15.5,16 C15.5,16.5522847 15.0522847,17 14.5,17 C13.9477153,17 13.5,16.5522847 13.5,16 L13.5,15.5 C13.5,13.290861 11.709139,11.5 9.5,11.5 L9,11.5 C8.44771525,11.5 8,11.0522847 8,10.5 C8,9.94771525 8.44771525,9.5 9,9.5 L9.5,9.5 C11.709139,9.5 13.5,7.709139 13.5,5.5 Z M4.5,12 C3.67157288,12 3,11.3284271 3,10.5 C3,9.67157288 3.67157288,9 4.5,9 C5.32842712,9 6,9.67157288 6,10.5 C6,11.3284271 5.32842712,12 4.5,12 Z M9,16 L10,16 C10.5522847,16 11,16.4477153 11,17 C11,17.5522847 10.5522847,18 10,18 L9,18 L9,19 C9,19.5522847 8.55228475,20 8,20 C7.44771525,20 7,19.5522847 7,19 L7,18 L6,18 C5.44771525,18 5,17.5522847 5,17 C5,16.4477153 5.44771525,16 6,16 L7,16 L7,15 C7,14.4477153 7.44771525,14 8,14 C8.55228475,14 9,14.4477153 9,15 L9,16 Z M12.8177951,10.5 C13.4841844,10.9430742 14.0569258,11.5158156 14.5,12.1822049 C14.9430742,11.5158156 15.5158156,10.9430742 16.1822049,10.5 C15.5158156,10.0569258 14.9430742,9.48418437 14.5,8.81779512 C14.0569258,9.48418437 13.4841844,10.0569258 12.8177951,10.5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Top experience</span></div><ul id="special-offers-list" class="_3xeXFm_LD6SL449SwaQeIk"><li class="LG3S6X7iyYg5Ejlp-bCZz">HIVE by Shangri-La &amp; Belvedere</li></ul></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="5412" data-marketing="true" data-index="54" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/hys-steakhouse-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/27064655.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/27064655.jpg 105w, https://images.otstatic.com/prod/27064655/2/small.jpg 160w, https://images.otstatic.com/prod/27064655/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/27064655.jpg" alt="A photo of Hy's Steakhouse - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/hys-steakhouse-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Hy's Steakhouse - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Hy's Steakhouse - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/hys-steakhouse-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Hy's Steakhouse - Toronto restaurant reviews (1784)">(1784)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 63 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=5412&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2041300253&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Hy's Steakhouse - Toronto restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=5412&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=102175748&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Hy's Steakhouse - Toronto restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="7527" data-marketing="true" data-index="55" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/jump-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23696094.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23696094.jpg 105w, https://images.otstatic.com/prod/23696094/1/small.jpg 160w, https://images.otstatic.com/prod/23696094/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23696094.jpg" alt="A photo of Jump Restaurant restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/jump-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Jump Restaurant restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Jump Restaurant</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/jump-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Jump Restaurant restaurant reviews (3061)">(3061)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary American • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 51 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=7527&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1057291132&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Jump Restaurant restaurant" aria-describedby="tooltip-1057291132">6:30 PM*<div id="tooltip-1057291132" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=7527&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2507254238&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Jump Restaurant restaurant" aria-describedby="tooltip-2507254238">6:45 PM*<div id="tooltip-2507254238" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=7527&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3172529523&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Jump Restaurant restaurant" aria-describedby="tooltip-3172529523">7:00 PM*<div id="tooltip-3172529523" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=7527&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3866089946&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Jump Restaurant restaurant" aria-describedby="tooltip-3866089946">7:15 PM*<div id="tooltip-3866089946" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=7527&amp;d=2021-08-06T19%3A45%3A00&amp;sd=2021-08-06T19%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3484513279&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:45 PM Reserve table at Jump Restaurant restaurant" aria-describedby="tooltip-3484513279">7:45 PM*<div id="tooltip-3484513279" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="347497" data-marketing="true" data-index="56" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/sud-forno-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42309608.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42309608.jpg 105w, https://images.otstatic.com/prod1/42309608/3/small.jpg 160w, https://images.otstatic.com/prod1/42309608/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42309608.jpg" alt="A photo of Sud Forno restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/sud-forno-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Sud Forno restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Sud Forno</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/sud-forno-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Sud Forno restaurant reviews (496)">(496)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 122 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=347497&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1815087523&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:15 PM Reserve table at Sud Forno restaurant">9:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1050571" data-marketing="true" data-index="57" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/la-pizza-and-la-pasta-eataly-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/5/28943464.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/5/28943464.jpg 105w, https://images.otstatic.com/prod1/28943464/5/small.jpg 160w, https://images.otstatic.com/prod1/28943464/5/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/5/28943464.jpg" alt="A photo of La Pizza &amp; La Pasta - Eataly Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/la-pizza-and-la-pasta-eataly-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="La Pizza &amp; La Pasta - Eataly Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">La Pizza &amp; La Pasta - Eataly Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.2 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.2</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/la-pizza-and-la-pasta-eataly-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview La Pizza &amp; La Pasta - Eataly Toronto restaurant reviews (238)">(238)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 91 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1050571&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1896262374&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at La Pizza &amp; La Pasta - Eataly Toronto restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1050571&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=234990338&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at La Pizza &amp; La Pasta - Eataly Toronto restaurant">5:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="226576" data-marketing="true" data-index="58" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/bella-vista-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/26057897.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/26057897.jpg 105w, https://images.otstatic.com/prod/26057897/1/small.jpg 160w, https://images.otstatic.com/prod/26057897/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/26057897.jpg" alt="A photo of Bella Vista restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/bella-vista-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Bella Vista restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Bella Vista</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.2 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.2</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/bella-vista-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Bella Vista restaurant reviews (21)">(21)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Little Italy / College St.</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 2 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=226576&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1075999116&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Bella Vista restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=226576&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1223797422&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Bella Vista restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=226576&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3122382843&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Bella Vista restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=226576&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3421262449&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Bella Vista restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=226576&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1242387904&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Bella Vista restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="73660" data-marketing="true" data-index="59" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/modus-ristorante?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23695552.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23695552.jpg 105w, https://images.otstatic.com/prod/23695552/1/small.jpg 160w, https://images.otstatic.com/prod/23695552/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23695552.jpg" alt="A photo of Modus Ristorante restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/modus-ristorante?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Modus Ristorante restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Modus Ristorante</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/modus-ristorante?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Modus Ristorante restaurant reviews (478)">(478)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 22 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=73660&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2015556780&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Modus Ristorante restaurant" aria-describedby="tooltip-2015556780">6:30 PM*<div id="tooltip-2015556780" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=73660&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4150367693&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Modus Ristorante restaurant" aria-describedby="tooltip-4150367693">6:45 PM*<div id="tooltip-4150367693" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=73660&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3751103634&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Modus Ristorante restaurant" aria-describedby="tooltip-3751103634">7:00 PM*<div id="tooltip-3751103634" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=73660&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1314850182&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Modus Ristorante restaurant" aria-describedby="tooltip-1314850182">7:15 PM*<div id="tooltip-1314850182" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=73660&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1394481625&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Modus Ristorante restaurant" aria-describedby="tooltip-1394481625">7:30 PM*<div id="tooltip-1394481625" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1113472" data-marketing="true" data-index="60" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/the-rabbit-hole-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42300918.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42300918.jpg 105w, https://images.otstatic.com/prod1/42300918/3/small.jpg 160w, https://images.otstatic.com/prod1/42300918/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42300918.jpg" alt="A photo of The Rabbit Hole restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/the-rabbit-hole-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Rabbit Hole restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Rabbit Hole</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/the-rabbit-hole-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Rabbit Hole restaurant reviews (52)">(52)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary British • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 25 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1113472&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1613389671&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at The Rabbit Hole restaurant" aria-describedby="tooltip-1613389671">4:30 PM*<div id="tooltip-1613389671" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1027057" data-marketing="true" data-index="61" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/parallel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/42468625.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/42468625.jpg 105w, https://images.otstatic.com/prod1/42468625/2/small.jpg 160w, https://images.otstatic.com/prod1/42468625/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/42468625.jpg" alt="A photo of Parallel restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/parallel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Parallel restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Parallel</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/parallel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Parallel restaurant reviews (304)">(304)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Middle Eastern • The Junction</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 24 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1027057&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=821444845&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at Parallel restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1027057&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1024191067&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at Parallel restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1075825" data-marketing="true" data-index="62" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/cabana-waterfront-patio-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/31016943.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/31016943.jpg 105w, https://images.otstatic.com/prod1/31016943/3/small.jpg 160w, https://images.otstatic.com/prod1/31016943/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/31016943.jpg" alt="A photo of Cabana Waterfront Patio restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/cabana-waterfront-patio-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Cabana Waterfront Patio restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Cabana Waterfront Patio</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.2 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.2</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/cabana-waterfront-patio-restaurant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Cabana Waterfront Patio restaurant reviews (980)">(980)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Distillery District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 519 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1075825&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4104045218&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Cabana Waterfront Patio restaurant" aria-describedby="tooltip-4104045218">6:45 PM*<div id="tooltip-4104045218" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="172300" data-marketing="true" data-index="63" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/the-keg-steakhouse-and-bar-mansion?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/30103909.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/30103909.jpg 105w, https://images.otstatic.com/prod1/30103909/2/small.jpg 160w, https://images.otstatic.com/prod1/30103909/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/30103909.jpg" alt="A photo of The Keg Steakhouse + Bar - Mansion restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/the-keg-steakhouse-and-bar-mansion?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Keg Steakhouse + Bar - Mansion restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Keg Steakhouse + Bar - Mansion</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/the-keg-steakhouse-and-bar-mansion?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Keg Steakhouse + Bar - Mansion restaurant reviews (1362)">(1362)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Cabbagetown</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 88 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=172300&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1785461895&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at The Keg Steakhouse + Bar - Mansion restaurant" aria-describedby="tooltip-1785461895">4:30 PM*<div id="tooltip-1785461895" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=172300&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3237766735&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at The Keg Steakhouse + Bar - Mansion restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="82957" data-marketing="true" data-index="64" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-marketing-ids="7" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/the-sultans-tent?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23698302.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23698302.jpg 105w, https://images.otstatic.com/prod/23698302/1/small.jpg 160w, https://images.otstatic.com/prod/23698302/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23698302.jpg" alt="A photo of The Sultan's Tent restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/the-sultans-tent?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Sultan's Tent restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Sultan's Tent</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/the-sultans-tent?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Sultan's Tent restaurant reviews (1036)">(1036)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Moroccan • St. Lawrence Market</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 55 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=82957&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=1000&amp;pofids=&amp;hash=2134454974&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo3LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard&amp;i=1" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at The Sultan's Tent restaurant and redeem +1,000 pts" aria-describedby="tooltip-2134454974">5:30 PM*<div id="tooltip-2134454974" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">+1,000 pts</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=82957&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=1000&amp;pofids=&amp;hash=2086378764&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo3LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard&amp;i=1" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at The Sultan's Tent restaurant and redeem +1,000 pts" aria-describedby="tooltip-2086378764">5:45 PM*<div id="tooltip-2086378764" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li></ul></div></a><span class="_3Uddh-sSTnRl3jenv0v4jU">+1,000 pts</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=82957&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2287257349&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoyODkzMjYsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at The Sultan's Tent restaurant" aria-describedby="tooltip-2287257349">9:15 PM*<div id="tooltip-2287257349" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=82957&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2821130945&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoyODkzMjYsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at The Sultan's Tent restaurant" aria-describedby="tooltip-2821130945">9:30 PM*<div id="tooltip-2821130945" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="115093" data-marketing="true" data-index="65" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/farmhouse-tavern-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/24972836.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/24972836.jpg 105w, https://images.otstatic.com/prod/24972836/1/small.jpg 160w, https://images.otstatic.com/prod/24972836/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/24972836.jpg" alt="A photo of FARMHOUSE Tavern restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/farmhouse-tavern-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="FARMHOUSE Tavern restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">FARMHOUSE Tavern</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/farmhouse-tavern-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview FARMHOUSE Tavern restaurant reviews (673)">(673)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • The Junction</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 48 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=115093&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3346845231&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at FARMHOUSE Tavern restaurant">9:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=115093&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=495152361&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at FARMHOUSE Tavern restaurant" aria-describedby="tooltip-495152361">9:30 PM*<div id="tooltip-495152361" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="79831" data-marketing="true" data-index="66" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/miller-tavern-downtown?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/26272572.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/26272572.jpg 105w, https://images.otstatic.com/prod/26272572/3/small.jpg 160w, https://images.otstatic.com/prod/26272572/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/26272572.jpg" alt="A photo of Miller Tavern - Downtown restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/miller-tavern-downtown?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Miller Tavern - Downtown restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Miller Tavern - Downtown</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.4</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/miller-tavern-downtown?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Miller Tavern - Downtown restaurant reviews (572)">(572)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steak • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 43 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=79831&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3865246724&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Miller Tavern - Downtown restaurant" aria-describedby="tooltip-3865246724">5:15 PM*<div id="tooltip-3865246724" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=79831&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4256449705&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Miller Tavern - Downtown restaurant" aria-describedby="tooltip-4256449705">5:30 PM*<div id="tooltip-4256449705" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=79831&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3633410756&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:15 PM Reserve table at Miller Tavern - Downtown restaurant" aria-describedby="tooltip-3633410756">9:15 PM*<div id="tooltip-3633410756" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=79831&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=23012169&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:30 PM Reserve table at Miller Tavern - Downtown restaurant" aria-describedby="tooltip-23012169">9:30 PM*<div id="tooltip-23012169" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="12229" data-marketing="true" data-index="67" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/biffs-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23890513.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23890513.jpg 105w, https://images.otstatic.com/prod/23890513/1/small.jpg 160w, https://images.otstatic.com/prod/23890513/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23890513.jpg" alt="A photo of Biff's Bistro restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/biffs-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Biff's Bistro restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Biff's Bistro</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/biffs-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Biff's Bistro restaurant reviews (3005)">(3005)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • French • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 31 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=12229&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=64521294&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Biff's Bistro restaurant" aria-describedby="tooltip-64521294">4:45 PM*<div id="tooltip-64521294" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=12229&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3643332384&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Biff's Bistro restaurant" aria-describedby="tooltip-3643332384">5:00 PM*<div id="tooltip-3643332384" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1035682" data-marketing="true" data-index="68" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/eighteen30-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/26113711.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/26113711.jpg 105w, https://images.otstatic.com/prod/26113711/1/small.jpg 160w, https://images.otstatic.com/prod/26113711/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/26113711.jpg" alt="A photo of eighteen30 restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/eighteen30-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="eighteen30 restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">eighteen30</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="3.1 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">3.1</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Good</span><a href="https://www.opentable.com/r/eighteen30-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview eighteen30 restaurant reviews (123)">(123)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 32 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1035682&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=407453247&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at eighteen30 restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1035682&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=104237821&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at eighteen30 restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1035682&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=137660533&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at eighteen30 restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1035682&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=870896317&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at eighteen30 restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1035682&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2227769071&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at eighteen30 restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="3877" data-marketing="true" data-index="69" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/ruths-chris-steak-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23684424.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23684424.jpg 105w, https://images.otstatic.com/prod/23684424/1/small.jpg 160w, https://images.otstatic.com/prod/23684424/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23684424.jpg" alt="A photo of Ruth's Chris Steak House - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/ruths-chris-steak-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Ruth's Chris Steak House - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Ruth's Chris Steak House - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/ruths-chris-steak-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Ruth's Chris Steak House - Toronto restaurant reviews (2093)">(2093)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 27 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=3877&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1802530671&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:15 PM Reserve table at Ruth's Chris Steak House - Toronto restaurant">6:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=3877&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1937904106&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Ruth's Chris Steak House - Toronto restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="82399" data-marketing="true" data-index="70" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/cafe-boulud?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/24005600.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/24005600.jpg 105w, https://images.otstatic.com/prod/24005600/1/small.jpg 160w, https://images.otstatic.com/prod/24005600/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/24005600.jpg" alt="A photo of Café Boulud restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/cafe-boulud?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Café Boulud restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Café Boulud</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/cafe-boulud?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Café Boulud restaurant reviews (2587)">(2587)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • French • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 57 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=82399&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1837821618&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Café Boulud restaurant">5:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=82399&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=585462671&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Café Boulud restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=82399&amp;d=2021-08-06T20%3A30%3A00&amp;sd=2021-08-06T20%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=306422643&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:30 PM Reserve table at Café Boulud restaurant">8:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=82399&amp;d=2021-08-06T20%3A45%3A00&amp;sd=2021-08-06T20%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2874110909&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:45 PM Reserve table at Café Boulud restaurant">8:45 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="995107" data-marketing="true" data-index="71" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/aanch-modernistic-indian-cuisine-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25936535.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25936535.jpg 105w, https://images.otstatic.com/prod/25936535/1/small.jpg 160w, https://images.otstatic.com/prod/25936535/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25936535.jpg" alt="A photo of Aanch-Modernistic Indian Cuisine restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/aanch-modernistic-indian-cuisine-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Aanch-Modernistic Indian Cuisine restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Aanch-Modernistic Indian Cuisine</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/aanch-modernistic-indian-cuisine-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Aanch-Modernistic Indian Cuisine restaurant reviews (174)">(174)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Indian • Downtown / Entertainment District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 3 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=995107&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=826338600&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Aanch-Modernistic Indian Cuisine restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=995107&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=413350341&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Aanch-Modernistic Indian Cuisine restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=995107&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=935330076&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Aanch-Modernistic Indian Cuisine restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=995107&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3180296512&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Aanch-Modernistic Indian Cuisine restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=995107&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2007506697&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Aanch-Modernistic Indian Cuisine restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="3178" data-marketing="true" data-index="72" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/mortons-the-steakhouse-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42275979.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42275979.jpg 105w, https://images.otstatic.com/prod1/42275979/3/small.jpg 160w, https://images.otstatic.com/prod1/42275979/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42275979.jpg" alt="A photo of Morton's The Steakhouse - Toronto restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/mortons-the-steakhouse-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Morton's The Steakhouse - Toronto restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Morton's The Steakhouse - Toronto</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/mortons-the-steakhouse-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Morton's The Steakhouse - Toronto restaurant reviews (1433)">(1433)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 35 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=3178&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4013443464&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Morton's The Steakhouse - Toronto restaurant" aria-describedby="tooltip-4013443464">5:30 PM*<div id="tooltip-4013443464" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=3178&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=604848329&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at Morton's The Steakhouse - Toronto restaurant">5:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=3178&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=915588570&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Morton's The Steakhouse - Toronto restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=3178&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2784867872&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Morton's The Steakhouse - Toronto restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=3178&amp;d=2021-08-06T21%3A15%3A00&amp;sd=2021-08-06T21%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4040396431&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Morton's The Steakhouse - Toronto restaurant" aria-describedby="tooltip-4040396431">9:15 PM*<div id="tooltip-4040396431" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="995428" data-marketing="true" data-index="73" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/fresh-on-front-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/31778128.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/31778128.jpg 105w, https://images.otstatic.com/prod1/31778128/1/small.jpg 160w, https://images.otstatic.com/prod1/31778128/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/31778128.jpg" alt="A photo of Fresh on Front restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/fresh-on-front-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Fresh on Front restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Fresh on Front</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/fresh-on-front-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Fresh on Front restaurant reviews (468)">(468)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Vegetarian / Vegan • St. Lawrence Market</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 35 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=995428&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3671171931&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Fresh on Front restaurant" aria-describedby="tooltip-3671171931">6:30 PM*<div id="tooltip-3671171931" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=995428&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2781909833&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Fresh on Front restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=995428&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2326387170&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Fresh on Front restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=995428&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4029096345&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Fresh on Front restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1188007" data-marketing="true" data-index="74" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/death-and-taxes-free-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><div class="tWVtceevqH0M323PmZfD5"><span class="_3OSkpovaXsWiFolf8qNGg3 _3tauplujEm_fVkvNSlWvv9 _2nSd6f-T-RwBgBd3WDNev4 _3ybmLecSinHUifiTEU61OR"><img class="_3563254ol9dkRoGXn_lru0" src="//cdn.otstatic.com/cfe/6/images/ic_photos-085592.svg" alt="A photo of Death &amp; Taxes Free House restaurant" title="" data-test=""></span></div></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/death-and-taxes-free-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Death &amp; Taxes Free House restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Death &amp; Taxes Free House</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="0 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div></div><a href="https://www.opentable.com/r/death-and-taxes-free-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Death &amp; Taxes Free House restaurant reviews (0)">(0)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Thornbury</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 1 time today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1188007&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=208598495&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=highTop&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Death &amp; Taxes Free House restaurant" aria-describedby="tooltip-208598495">6:30 PM*<div id="tooltip-208598495" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1188007&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3569301643&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=highTop&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Death &amp; Taxes Free House restaurant" aria-describedby="tooltip-3569301643">6:45 PM*<div id="tooltip-3569301643" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>High Top</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1014310" data-marketing="true" data-index="75" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/the-roof-at-soco-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25622265.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25622265.jpg 105w, https://images.otstatic.com/prod/25622265/1/small.jpg 160w, https://images.otstatic.com/prod/25622265/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25622265.jpg" alt="A photo of The Roof at SOCO restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/the-roof-at-soco-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Roof at SOCO restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Roof at SOCO</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.0</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Excellent</span><a href="https://www.opentable.com/r/the-roof-at-soco-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Roof at SOCO restaurant reviews (23)">(23)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Downtown / Entertainment District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 46 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1014310&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1091578402&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at The Roof at SOCO restaurant" aria-describedby="tooltip-1091578402">5:30 PM*<div id="tooltip-1091578402" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1014310&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3641968236&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at The Roof at SOCO restaurant" aria-describedby="tooltip-3641968236">5:45 PM*<div id="tooltip-3641968236" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1046506" data-marketing="true" data-index="76" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/pizza-e-pazzi-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/26451207.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/26451207.jpg 105w, https://images.otstatic.com/prod/26451207/2/small.jpg 160w, https://images.otstatic.com/prod/26451207/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/26451207.jpg" alt="A photo of Pizza e Pazzi restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/pizza-e-pazzi-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Pizza e Pazzi restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Pizza e Pazzi</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/pizza-e-pazzi-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Pizza e Pazzi restaurant reviews (140)">(140)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • St. Clair West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 17 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1046506&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2872601299&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Pizza e Pazzi restaurant" aria-describedby="tooltip-2872601299">5:30 PM*<div id="tooltip-2872601299" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1046506&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=617533915&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Pizza e Pazzi restaurant" aria-describedby="tooltip-617533915">6:00 PM*<div id="tooltip-617533915" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>High Top</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="147460" data-marketing="true" data-index="77" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/sorrel-rosedale?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25117080.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25117080.jpg 105w, https://images.otstatic.com/prod/25117080/1/small.jpg 160w, https://images.otstatic.com/prod/25117080/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25117080.jpg" alt="A photo of Sorrel Rosedale restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/sorrel-rosedale?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Sorrel Rosedale restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Sorrel Rosedale</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/sorrel-rosedale?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Sorrel Rosedale restaurant reviews (659)">(659)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary European • Rosedale / Summerhill</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 26 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=147460&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=495972700&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Sorrel Rosedale restaurant" aria-describedby="tooltip-495972700">9:00 PM*<div id="tooltip-495972700" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1066069" data-marketing="true" data-index="78" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/hendriks-restaurant-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/28974492.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/28974492.jpg 105w, https://images.otstatic.com/prod1/28974492/2/small.jpg 160w, https://images.otstatic.com/prod1/28974492/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/28974492.jpg" alt="A photo of Hendriks Restaurant &amp; Bar restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/hendriks-restaurant-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Hendriks Restaurant &amp; Bar restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Hendriks Restaurant &amp; Bar</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/hendriks-restaurant-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Hendriks Restaurant &amp; Bar restaurant reviews (21)">(21)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary American • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 19 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1066069&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3207070262&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Hendriks Restaurant &amp; Bar restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1066069&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1357440221&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Hendriks Restaurant &amp; Bar restaurant">5:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="78649" data-marketing="true" data-index="79" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/michaels-on-simcoe?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/28526926.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/28526926.jpg 105w, https://images.otstatic.com/prod/28526926/2/small.jpg 160w, https://images.otstatic.com/prod/28526926/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/28526926.jpg" alt="A photo of Michael's on Simcoe restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/michaels-on-simcoe?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Michael's on Simcoe restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Michael's on Simcoe</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/michaels-on-simcoe?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Michael's on Simcoe restaurant reviews (1047)">(1047)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Downtown / Entertainment District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 20 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=78649&amp;d=2021-08-06T17%3A45%3A00&amp;sd=2021-08-06T17%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2462498596&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:45 PM Reserve table at Michael's on Simcoe restaurant">5:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=78649&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=338689053&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Michael's on Simcoe restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=78649&amp;d=2021-08-06T19%3A45%3A00&amp;sd=2021-08-06T19%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3460088539&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:45 PM Reserve table at Michael's on Simcoe restaurant">7:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="262789" data-marketing="true" data-index="80" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/gyu-kaku-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25275042.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25275042.jpg 105w, https://images.otstatic.com/prod/25275042/1/small.jpg 160w, https://images.otstatic.com/prod/25275042/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25275042.jpg" alt="A photo of Gyu-Kaku - Toronto, ON (Downtown) restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/gyu-kaku-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Gyu-Kaku - Toronto, ON (Downtown) restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Gyu-Kaku - Toronto, ON (Downtown)</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.9 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.9</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/gyu-kaku-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Gyu-Kaku - Toronto, ON (Downtown) restaurant reviews (374)">(374)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Japanese • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 26 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=262789&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1115632786&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Gyu-Kaku - Toronto, ON (Downtown) restaurant">5:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=262789&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=489285487&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Gyu-Kaku - Toronto, ON (Downtown) restaurant">5:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="87505" data-marketing="true" data-index="81" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-marketing-ids="1413261" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/pizza-rustica-restaurant-and-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23699209.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23699209.jpg 105w, https://images.otstatic.com/prod/23699209/1/small.jpg 160w, https://images.otstatic.com/prod/23699209/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23699209.jpg" alt="A photo of Pizza Rustica Restaurant &amp; Bar restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/pizza-rustica-restaurant-and-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Pizza Rustica Restaurant &amp; Bar restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Pizza Rustica Restaurant &amp; Bar</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="3.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">3.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Very Good</span><a href="https://www.opentable.com/pizza-rustica-restaurant-and-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Pizza Rustica Restaurant &amp; Bar restaurant reviews (159)">(159)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Entertainment District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 4 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=87505&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1788322410&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoyODQ5NDYsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Pizza Rustica Restaurant &amp; Bar restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=87505&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1291729515&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoyODQ5NDYsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Pizza Rustica Restaurant &amp; Bar restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=87505&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2620307154&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoyODQ5NDYsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Pizza Rustica Restaurant &amp; Bar restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=87505&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1634371301&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoyODQ5NDYsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Pizza Rustica Restaurant &amp; Bar restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=87505&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3030856542&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoyODQ5NDYsInMiOjAsIm4iOjB9&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Pizza Rustica Restaurant &amp; Bar restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="115324" data-marketing="true" data-index="82" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/gatsby-by-the-windsor-arms-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/31967129.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/31967129.jpg 105w, https://images.otstatic.com/prod1/31967129/3/small.jpg 160w, https://images.otstatic.com/prod1/31967129/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/31967129.jpg" alt="A photo of Gatsby by Windsor Arms restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/gatsby-by-the-windsor-arms-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Gatsby by Windsor Arms restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Gatsby by Windsor Arms</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="3.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">3.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Excellent</span><a href="https://www.opentable.com/r/gatsby-by-the-windsor-arms-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Gatsby by Windsor Arms restaurant reviews (209)">(209)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary American • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 11 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=115324&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1565418999&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Gatsby by Windsor Arms restaurant" aria-describedby="tooltip-1565418999">5:30 PM*<div id="tooltip-1565418999" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=115324&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2814008089&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Gatsby by Windsor Arms restaurant" aria-describedby="tooltip-2814008089">6:00 PM*<div id="tooltip-2814008089" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="47410" data-marketing="true" data-index="83" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/lai-wah-heen-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23687078.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23687078.jpg 105w, https://images.otstatic.com/prod/23687078/1/small.jpg 160w, https://images.otstatic.com/prod/23687078/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23687078.jpg" alt="A photo of Lai Wah Heen Restaurant restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/lai-wah-heen-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Lai Wah Heen Restaurant restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Lai Wah Heen Restaurant</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/lai-wah-heen-restaurant?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Lai Wah Heen Restaurant restaurant reviews (893)">(893)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Chinese • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 15 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=47410&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3795141122&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Lai Wah Heen Restaurant restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=47410&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3717337104&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Lai Wah Heen Restaurant restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=47410&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4105185278&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Lai Wah Heen Restaurant restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=47410&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4222315586&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Lai Wah Heen Restaurant restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=47410&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2042105152&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Lai Wah Heen Restaurant restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1200643" data-marketing="true" data-index="84" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/misty-restaurant-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><div class="tWVtceevqH0M323PmZfD5"><span class="_3OSkpovaXsWiFolf8qNGg3 _3tauplujEm_fVkvNSlWvv9 _2nSd6f-T-RwBgBd3WDNev4 _3ybmLecSinHUifiTEU61OR"><img class="_3563254ol9dkRoGXn_lru0" src="//cdn.otstatic.com/cfe/6/images/ic_photos-085592.svg" alt="A photo of Misty Restaurant &amp; Bar restaurant" title="" data-test=""></span></div></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/misty-restaurant-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Misty Restaurant &amp; Bar restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Misty Restaurant &amp; Bar</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/misty-restaurant-and-bar-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Misty Restaurant &amp; Bar restaurant reviews (3)">(3)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary European • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 10 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1200643&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=98347042&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at Misty Restaurant &amp; Bar restaurant">6:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1200643&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1808660721&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Misty Restaurant &amp; Bar restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1200643&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1541500298&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Misty Restaurant &amp; Bar restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1200643&amp;d=2021-08-06T19%3A45%3A00&amp;sd=2021-08-06T19%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2969023381&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:45 PM Reserve table at Misty Restaurant &amp; Bar restaurant">7:45 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="984973" data-marketing="true" data-index="85" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/rooftop-bar-at-the-broadview-hotel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25068394.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25068394.jpg 105w, https://images.otstatic.com/prod/25068394/1/small.jpg 160w, https://images.otstatic.com/prod/25068394/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25068394.jpg" alt="A photo of Rooftop Bar at the Broadview Hotel restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/rooftop-bar-at-the-broadview-hotel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Rooftop Bar at the Broadview Hotel restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Rooftop Bar at the Broadview Hotel</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/rooftop-bar-at-the-broadview-hotel-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Rooftop Bar at the Broadview Hotel restaurant reviews (756)">(756)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Global, International • Leslieville / Riverdale</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 82 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=984973&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=649310854&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Rooftop Bar at the Broadview Hotel restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=984973&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4113010242&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Rooftop Bar at the Broadview Hotel restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=984973&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=251872533&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Rooftop Bar at the Broadview Hotel restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=984973&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2702030678&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Rooftop Bar at the Broadview Hotel restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=984973&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2367710341&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Rooftop Bar at the Broadview Hotel restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1066009" data-marketing="true" data-index="86" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/uncle-tonys-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/14/28918537.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/14/28918537.jpg 105w, https://images.otstatic.com/prod1/28918537/14/small.jpg 160w, https://images.otstatic.com/prod1/28918537/14/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/14/28918537.jpg" alt="A photo of Uncle Tony's restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/uncle-tonys-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Uncle Tony's restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Uncle Tony's</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.0</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Excellent</span><a href="https://www.opentable.com/r/uncle-tonys-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Uncle Tony's restaurant reviews (42)">(42)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 8 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1066009&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3238115462&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Uncle Tony's restaurant" aria-describedby="tooltip-3238115462">6:30 PM*<div id="tooltip-3238115462" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1066009&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2692573741&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Uncle Tony's restaurant" aria-describedby="tooltip-2692573741">6:45 PM*<div id="tooltip-2692573741" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1066009&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4193029291&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Uncle Tony's restaurant" aria-describedby="tooltip-4193029291">7:00 PM*<div id="tooltip-4193029291" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1066009&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2267289036&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Uncle Tony's restaurant" aria-describedby="tooltip-2267289036">7:15 PM*<div id="tooltip-2267289036" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1066009&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2750446609&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Uncle Tony's restaurant" aria-describedby="tooltip-2750446609">7:30 PM*<div id="tooltip-2750446609" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Bar</li><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1037356" data-marketing="true" data-index="87" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/mineral-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/27352892.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/27352892.jpg 105w, https://images.otstatic.com/prod/27352892/3/small.jpg 160w, https://images.otstatic.com/prod/27352892/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/27352892.jpg" alt="A photo of Mineral restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/mineral-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Mineral restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Mineral</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.9 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.9</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/mineral-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Mineral restaurant reviews (61)">(61)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$$<span class="_1s81GB3gesncA-230LO-_w"></span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Filipino • Rosedale / Summerhill</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 3 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1037356&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=597540113&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Mineral restaurant" aria-describedby="tooltip-597540113">5:15 PM*<div id="tooltip-597540113" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1037356&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3033364035&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo2LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at Mineral restaurant" aria-describedby="tooltip-3033364035">5:30 PM*<div id="tooltip-3033364035" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="330406" data-marketing="true" data-index="88" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/smoque-n-bones-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25163613.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25163613.jpg 105w, https://images.otstatic.com/prod/25163613/1/small.jpg 160w, https://images.otstatic.com/prod/25163613/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25163613.jpg" alt="A photo of Smoque N Bones restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/smoque-n-bones-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Smoque N Bones restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Smoque N Bones</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/smoque-n-bones-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Smoque N Bones restaurant reviews (447)">(447)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Barbecue • Queen West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 12 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=330406&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2541881390&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Smoque N Bones restaurant" aria-describedby="tooltip-2541881390">5:15 PM*<div id="tooltip-2541881390" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=330406&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2745195134&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Smoque N Bones restaurant" aria-describedby="tooltip-2745195134">5:30 PM*<div id="tooltip-2745195134" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=330406&amp;d=2021-08-06T20%3A45%3A00&amp;sd=2021-08-06T20%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=417173531&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:45 PM Reserve table at Smoque N Bones restaurant" aria-describedby="tooltip-417173531">8:45 PM*<div id="tooltip-417173531" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=330406&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2357731627&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at Smoque N Bones restaurant" aria-describedby="tooltip-2357731627">9:00 PM*<div id="tooltip-2357731627" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1201720" data-marketing="true" data-index="89" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/my-green-merchant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/2/42598014.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/2/42598014.jpg 105w, https://images.otstatic.com/prod1/42598014/2/small.jpg 160w, https://images.otstatic.com/prod1/42598014/2/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/2/42598014.jpg" alt="A photo of My Green Merchant restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/my-green-merchant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="My Green Merchant restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">My Green Merchant</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="0 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div></div><a href="https://www.opentable.com/r/my-green-merchant-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview My Green Merchant restaurant reviews (0)">(0)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Cocktail Bar • Bloor West Village</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 2 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1201720&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2421769925&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjczNjkxLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=37255&amp;ra=&amp;isMandatory=false&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at My Green Merchant restaurant and redeem Experiences">5:00 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1201720&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2615769381&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjczNjkxLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=37255&amp;ra=&amp;isMandatory=false&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at My Green Merchant restaurant and redeem Experiences">6:00 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1201720&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2049366229&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjczNjkxLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=37255&amp;ra=&amp;isMandatory=false&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at My Green Merchant restaurant and redeem Experiences">7:00 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1201720&amp;d=2021-08-06T20%3A00%3A00&amp;sd=2021-08-06T20%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3526083682&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjczNjkxLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=37255&amp;ra=&amp;isMandatory=false&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:00 PM Reserve table at My Green Merchant restaurant and redeem Experiences">8:00 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1201720&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2045458395&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjoxNjczNjkxLCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=37255&amp;ra=&amp;isMandatory=false&amp;st=Experience" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at My Green Merchant restaurant and redeem Experiences">9:00 PM</a><span class="_3Uddh-sSTnRl3jenv0v4jU">Experiences</span></li></ul><div class="LDZNZTbXAbWJPLySat9S8"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSpecials"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M13.5,5.5 L13.5,5 C13.5,4.44771525 13.9477153,4 14.5,4 C15.0522847,4 15.5,4.44771525 15.5,5 L15.5,5.5 C15.5,7.709139 17.290861,9.5 19.5,9.5 L20,9.5 C20.5522847,9.5 21,9.94771525 21,10.5 C21,11.0522847 20.5522847,11.5 20,11.5 L19.5,11.5 C17.290861,11.5 15.5,13.290861 15.5,15.5 L15.5,16 C15.5,16.5522847 15.0522847,17 14.5,17 C13.9477153,17 13.5,16.5522847 13.5,16 L13.5,15.5 C13.5,13.290861 11.709139,11.5 9.5,11.5 L9,11.5 C8.44771525,11.5 8,11.0522847 8,10.5 C8,9.94771525 8.44771525,9.5 9,9.5 L9.5,9.5 C11.709139,9.5 13.5,7.709139 13.5,5.5 Z M4.5,12 C3.67157288,12 3,11.3284271 3,10.5 C3,9.67157288 3.67157288,9 4.5,9 C5.32842712,9 6,9.67157288 6,10.5 C6,11.3284271 5.32842712,12 4.5,12 Z M9,16 L10,16 C10.5522847,16 11,16.4477153 11,17 C11,17.5522847 10.5522847,18 10,18 L9,18 L9,19 C9,19.5522847 8.55228475,20 8,20 C7.44771525,20 7,19.5522847 7,19 L7,18 L6,18 C5.44771525,18 5,17.5522847 5,17 C5,16.4477153 5.44771525,16 6,16 L7,16 L7,15 C7,14.4477153 7.44771525,14 8,14 C8.55228475,14 9,14.4477153 9,15 L9,16 Z M12.8177951,10.5 C13.4841844,10.9430742 14.0569258,11.5158156 14.5,12.1822049 C14.9430742,11.5158156 15.5158156,10.9430742 16.1822049,10.5 C15.5158156,10.0569258 14.9430742,9.48418437 14.5,8.81779512 C14.0569258,9.48418437 13.4841844,10.0569258 12.8177951,10.5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Top experience</span></div><ul id="special-offers-list" class="_3xeXFm_LD6SL449SwaQeIk"><li class="LG3S6X7iyYg5Ejlp-bCZz">Toronto's First Cannabis Cocktail Tasting!</li></ul></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1189723" data-marketing="true" data-index="90" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/shenanigans-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42346757.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42346757.jpg 105w, https://images.otstatic.com/prod1/42346757/3/small.jpg 160w, https://images.otstatic.com/prod1/42346757/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42346757.jpg" alt="A photo of Shenanigans restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/shenanigans-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Shenanigans restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Shenanigans</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.3 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1h89tIvK1Zm7zGJiJKLt-G _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.3</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/shenanigans-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Shenanigans restaurant reviews (38)">(38)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Pub • St. Clair West</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 24 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1189723&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2760892709&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Shenanigans restaurant" aria-describedby="tooltip-2760892709">5:15 PM*<div id="tooltip-2760892709" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1189723&amp;d=2021-08-06T17%3A30%3A00&amp;sd=2021-08-06T17%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4175048974&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:30 PM Reserve table at Shenanigans restaurant" aria-describedby="tooltip-4175048974">5:30 PM*<div id="tooltip-4175048974" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1189723&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3064981057&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at Shenanigans restaurant" aria-describedby="tooltip-3064981057">9:00 PM*<div id="tooltip-3064981057" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="213094" data-marketing="true" data-index="91" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/brownes-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/24166274.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/24166274.jpg 105w, https://images.otstatic.com/prod/24166274/1/small.jpg 160w, https://images.otstatic.com/prod/24166274/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/24166274.jpg" alt="A photo of Brownes Bistro restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/brownes-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Brownes Bistro restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Brownes Bistro</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/brownes-bistro?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Brownes Bistro restaurant reviews (226)">(226)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Contemporary French • Rosedale / Summerhill</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 3 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=213094&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1541976559&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at Brownes Bistro restaurant">6:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=213094&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1683914161&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Brownes Bistro restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=213094&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=98109329&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Brownes Bistro restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=213094&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2786358211&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Brownes Bistro restaurant">7:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=213094&amp;d=2021-08-06T20%3A00%3A00&amp;sd=2021-08-06T20%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3987109460&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:00 PM Reserve table at Brownes Bistro restaurant">8:00 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="53866" data-marketing="true" data-index="92" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/koko-share-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/24950201.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/24950201.jpg 105w, https://images.otstatic.com/prod/24950201/1/small.jpg 160w, https://images.otstatic.com/prod/24950201/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/24950201.jpg" alt="A photo of KOKO! Share Bar restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/koko-share-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="KOKO! Share Bar restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">KOKO! Share Bar</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.7 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.7</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/koko-share-bar?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview KOKO! Share Bar restaurant reviews (240)">(240)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Korean • Yorkville</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 17 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 1 timeslot left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=53866&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3829655042&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="Select table type for reservation at KOKO! Share Bar restaurant" aria-describedby="tooltip-3829655042">5:15 PM*<div id="tooltip-3829655042" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Standard</li><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="242542" data-marketing="true" data-index="93" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/the-queen-and-beaver-public-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25552553.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25552553.jpg 105w, https://images.otstatic.com/prod/25552553/1/small.jpg 160w, https://images.otstatic.com/prod/25552553/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25552553.jpg" alt="A photo of The Queen &amp; Beaver Public House restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/the-queen-and-beaver-public-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="The Queen &amp; Beaver Public House restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">The Queen &amp; Beaver Public House</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/the-queen-and-beaver-public-house-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview The Queen &amp; Beaver Public House restaurant reviews (187)">(187)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Pub • Yonge and Dundas Square</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 21 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=242542&amp;d=2021-08-06T18%3A00%3A00&amp;sd=2021-08-06T18%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2669426436&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:00 PM Reserve table at The Queen &amp; Beaver Public House restaurant">6:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=242542&amp;d=2021-08-06T18%3A15%3A00&amp;sd=2021-08-06T18%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3222788425&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:15 PM Reserve table at The Queen &amp; Beaver Public House restaurant">6:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=242542&amp;d=2021-08-06T21%3A00%3A00&amp;sd=2021-08-06T21%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3854955597&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:00 PM Reserve table at The Queen &amp; Beaver Public House restaurant">9:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="54079" data-marketing="true" data-index="94" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-marketing-ids="1677281" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/estiatorio-volos?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/24610856.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/24610856.jpg 105w, https://images.otstatic.com/prod/24610856/1/small.jpg 160w, https://images.otstatic.com/prod/24610856/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/24610856.jpg" alt="A photo of Estiatorio VOLOS restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/estiatorio-volos?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Estiatorio VOLOS restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Estiatorio VOLOS</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="3.9 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_3kO6rduhXv8PjZkMM0vdp5 _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">3.9</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Excellent</span><a href="https://www.opentable.com/estiatorio-volos?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Estiatorio VOLOS restaurant reviews (1836)">(1836)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Seafood • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 17 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=54079&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1218298010&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Estiatorio VOLOS restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=54079&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3155995658&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Estiatorio VOLOS restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=54079&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2231338744&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Estiatorio VOLOS restaurant">7:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=54079&amp;d=2021-08-06T19%3A45%3A00&amp;sd=2021-08-06T19%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1436647580&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:45 PM Reserve table at Estiatorio VOLOS restaurant">7:45 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1036222" data-marketing="true" data-index="95" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/napoli-centrale-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/26074504.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/26074504.jpg 105w, https://images.otstatic.com/prod/26074504/1/small.jpg 160w, https://images.otstatic.com/prod/26074504/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/26074504.jpg" alt="A photo of Napoli Centrale restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/napoli-centrale-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Napoli Centrale restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Napoli Centrale</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.6 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.6</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/napoli-centrale-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Napoli Centrale restaurant reviews (19)">(19)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Pizzeria • The Annex</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 1 time today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1036222&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3728222046&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Napoli Centrale restaurant">5:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1036222&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=504954505&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Napoli Centrale restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1036222&amp;d=2021-08-06T20%3A00%3A00&amp;sd=2021-08-06T20%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2470244724&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:00 PM Reserve table at Napoli Centrale restaurant">8:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1036222&amp;d=2021-08-06T21%3A30%3A00&amp;sd=2021-08-06T21%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1167235934&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="9:30 PM Reserve table at Napoli Centrale restaurant">9:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="239266" data-marketing="true" data-index="96" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/la-fenice-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/28094464.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/28094464.jpg 105w, https://images.otstatic.com/prod/28094464/3/small.jpg 160w, https://images.otstatic.com/prod/28094464/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/28094464.jpg" alt="A photo of La Fenice restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/la-fenice-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="La Fenice restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">La Fenice</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/la-fenice-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview La Fenice restaurant reviews (229)">(229)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Italian • Downtown / Entertainment District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 2 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 2 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=239266&amp;d=2021-08-06T16%3A30%3A00&amp;sd=2021-08-06T16%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3382500872&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:30 PM Reserve table at La Fenice restaurant">4:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=239266&amp;d=2021-08-06T16%3A45%3A00&amp;sd=2021-08-06T16%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=884734407&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="4:45 PM Reserve table at La Fenice restaurant">4:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="729979" data-marketing="true" data-index="97" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/amsterdam-brewhouse-and-restaurant-leaside-east-york?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/25166815.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/25166815.jpg 105w, https://images.otstatic.com/prod/25166815/1/small.jpg 160w, https://images.otstatic.com/prod/25166815/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/25166815.jpg" alt="A photo of Amsterdam Barrel House restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/amsterdam-brewhouse-and-restaurant-leaside-east-york?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Amsterdam Barrel House restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Amsterdam Barrel House</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/amsterdam-brewhouse-and-restaurant-leaside-east-york?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Amsterdam Barrel House restaurant reviews (494)">(494)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Comfort Food • Leaside</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 88 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=729979&amp;d=2021-08-06T17%3A00%3A00&amp;sd=2021-08-06T17%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1378338036&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:00 PM Reserve table at Amsterdam Barrel House restaurant" aria-describedby="tooltip-1378338036">5:00 PM*<div id="tooltip-1378338036" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=729979&amp;d=2021-08-06T17%3A15%3A00&amp;sd=2021-08-06T17%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2755749735&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="5:15 PM Reserve table at Amsterdam Barrel House restaurant" aria-describedby="tooltip-2755749735">5:15 PM*<div id="tooltip-2755749735" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=729979&amp;d=2021-08-06T20%3A30%3A00&amp;sd=2021-08-06T20%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4303025&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:30 PM Reserve table at Amsterdam Barrel House restaurant" aria-describedby="tooltip-4303025">8:30 PM*<div id="tooltip-4303025" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=729979&amp;d=2021-08-06T20%3A45%3A00&amp;sd=2021-08-06T20%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1386409639&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="8:45 PM Reserve table at Amsterdam Barrel House restaurant" aria-describedby="tooltip-1386409639">8:45 PM*<div id="tooltip-1386409639" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1197088" data-marketing="true" data-index="98" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/bar-le-germain-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/42510840.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/42510840.jpg 105w, https://images.otstatic.com/prod1/42510840/3/small.jpg 160w, https://images.otstatic.com/prod1/42510840/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/42510840.jpg" alt="A photo of Bar Le Germain restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/bar-le-germain-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Bar Le Germain restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Bar Le Germain</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/r/bar-le-germain-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Bar Le Germain restaurant reviews (4)">(4)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$<span class="_1s81GB3gesncA-230LO-_w">$$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Fusion / Eclectic • Harbourfront</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 7 times today</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1197088&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3779384342&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Bar Le Germain restaurant" aria-describedby="tooltip-3779384342">6:30 PM*<div id="tooltip-3779384342" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1197088&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3741547649&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Bar Le Germain restaurant" aria-describedby="tooltip-3741547649">6:45 PM*<div id="tooltip-3741547649" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=1197088&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=1910403055&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Bar Le Germain restaurant" aria-describedby="tooltip-1910403055">7:00 PM*<div id="tooltip-1910403055" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1197088&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=2421985837&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at Bar Le Germain restaurant" aria-describedby="tooltip-2421985837">7:15 PM*<div id="tooltip-2421985837" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1197088&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3721223427&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=outdoor&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at Bar Le Germain restaurant" aria-describedby="tooltip-3721223427">7:30 PM*<div id="tooltip-3721223427" class="_1nYmUrNby-IZJYFGZGq69V _1URj_b9y1JwhlXijbeQezT" role="tooltip"><h6 class="gQWMkT1JOvQWzdj32CoBJ">Seating options</h6><ul class="_2n4MdtDEBFbGZrXqApUC4P"><li>Outdoor</li></ul></div></a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="18358" data-marketing="true" data-index="99" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/bardis-steakhouse?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/1/23677495.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/1/23677495.jpg 105w, https://images.otstatic.com/prod/23677495/1/small.jpg 160w, https://images.otstatic.com/prod/23677495/1/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/1/23677495.jpg" alt="A photo of Bardi's Steakhouse restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/bardis-steakhouse?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="Bardi's Steakhouse restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">Bardi's Steakhouse</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.5 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_28k_iav1_Eiqo1_PHjcTNC _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.5</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Awesome</span><a href="https://www.opentable.com/bardis-steakhouse?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview Bardi's Steakhouse restaurant reviews (515)">(515)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Steakhouse • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 9 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 3 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=18358&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3767913524&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at Bardi's Steakhouse restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=18358&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4226313054&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at Bardi's Steakhouse restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><a href="https://www.opentable.com/book/validate?rid=18358&amp;d=2021-08-06T19%3A00%3A00&amp;sd=2021-08-06T19%3A00%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=165883625&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:00 PM Reserve table at Bardi's Steakhouse restaurant">7:00 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div><div class="_3uVfVbI1iLfMbszbU6KoOL" data-rid="1042294" data-marketing="true" data-index="100" data-avt="eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ" data-test="restaurant-card"><div class="_2WscsV4-12URxWrPI0TvSI" style="min-height: 72px;"><a href="https://www.opentable.com/r/reign-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" tabindex="-1" target="_blank"><img srcset="https://resizer.otstatic.com/v2/photos/legacy/3/26520443.jpg 208w, https://resizer.otstatic.com/v2/photos/xsmall/3/26520443.jpg 105w, https://images.otstatic.com/prod/26520443/3/small.jpg 160w, https://images.otstatic.com/prod/26520443/3/medium.jpg 320w" sizes="(min-width: 1056px) 205px, 90px" src="https://resizer.otstatic.com/v2/photos/legacy/3/26520443.jpg" alt="A photo of REIGN restaurant" class="_3OSkpovaXsWiFolf8qNGg3 _3E2qJsRw2vlsds_A_lLvxd" title="" data-test=""></a></div><div class="_17v75S2htyk61cW3_VXibj"><div class="_1liK37RaUN7lnBs9g1TPyp"><div class="_1HpaBjJWDwElwdkD3OKS7J"><a href="https://www.opentable.com/r/reign-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00" class="_1e9PcCDb012hY4BcGfraQB" data-test="res-card-name" aria-label="REIGN restaurant" target="_blank" rel="noopener noreferrer"><h6 class="k8o46Bca35RzHNtQFy3bH">REIGN</h6></a></div><div class="_23JVvFevCvj_mutWbEzTgU"><div class="_25xMNiLip-SZ1zuCkSA7ge"><div class="_2IMP-CK1uNe12eCxaNLqbL" aria-label="4.8 stars"><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_1J28HXkF9OXU_OaZfUmxou _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div><div class="_14TQM3HXE0NyIpx6aeIwn1 _2UvC6jGDOaDUaN4lDzrERj"><i class="_2A9R7IBMmNo8l86RU1SUeg _2XDQgnHhhnZspJEFGbowSx" aria-hidden="true" tab-index="-1" focusable="no"></i></div></div><div class="_2s6ofZ_eiTKuvNHV3mVnaG">4.8</div></div><span class="_3vuCAUg_pskwYkQfEGhnPy">Exceptional</span><a href="https://www.opentable.com/r/reign-toronto?corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&amp;p=2&amp;sd=2021-08-06T19%3A00%3A00#reviews" class="_37NqKSTDkDYtsJrAU287ai" aria-label="Preview REIGN restaurant reviews (210)">(210)</a></div><div class="_1Y-E7ZnLMoyE5mrwg00NBj"><span class="_3sSkv7iJ6Tl1VxRjYAjp13">$$$<span class="_1s81GB3gesncA-230LO-_w">$</span></span><div class="_2p0jcmKJSDjEh-wNrLIpzJ" data-test="cuisine-and-location"> • Canadian • Downtown / Financial District</div></div></div><div class="_3MpMHmAvA25QerTwmHNPxy"><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb" data-test="icSocialProof"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15.5,5 C15.2239,5 15,5.223846 15,5.5 L15,6.5 C15,6.77615 15.2239,7 15.5,7 L17.5858,7 L14,10.58578 L12.70711,9.29291 L12.35355,8.93933 C12.15829,8.74408 11.84171,8.74408 11.64645,8.93933 L11.29289,9.29291 L5,15.5858 L5,7 L11.5,7 C11.77614,7 12,6.77615 12,6.5 L12,5.5 C12,5.22385 11.77614,5 11.5,5 L5,5 C3.89543,5 3,5.89542 3,7 L3,17 C3,18.1046 3.89543,19 5,19 L19,19 C20.1046,19 21,18.1046 21,17 L21,14.5 C21,14.2238 20.7761,14 20.5,14 L19.5,14 C19.2239,14 19,14.2238 19,14.5 L19,17 L6.4142,17 L12,11.41422 L13.2929,12.70709 L13.6464,13.06067 C13.8417,13.25592 14.1583,13.25592 14.3536,13.06067 L14.7071,12.70709 L19,8.41422 L19,10.5 C19,10.77615 19.2239,11 19.5,11 L20.5,11 C20.7761,11 21,10.77615 21,10.5 L21,6 L21,5.5 C21,5.223846 20.7761,5 20.5,5 L20,5 L15.5,5 Z" fill="#2D333F" fill-rule="nonzero"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">Booked 13 times today</span></div><div class="_2Ux1qxjjShCp6Q_A4v8g7q"><span class="_1hUZ3xV2F64jWhaNdNV8RZ nMtWpRjXjtouDQV4aHiJb _3toesI7iAuWnPhuZol6cbR" data-test="icScarcity"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M15,5 C18.8659932,5 22,8.13400675 22,12 C22,15.8659932 18.8659932,19 15,19 C11.1340068,19 8,15.8659932 8,12 C8,8.13400675 11.1340068,5 15,5 Z M15,17 C17.7614237,17 20,14.7614237 20,12 C20,9.23857625 17.7614237,7 15,7 C12.2385763,7 10,9.23857625 10,12 C10,14.7614237 12.2385763,17 15,17 Z M17.5,11 C17.7761424,11 18,11.2238576 18,11.5 L18,12.5 C18,12.7761424 17.7761424,13 17.5,13 L14.5,13 C14.2238576,13 14,12.7761424 14,12.5 L14,8.5 C14,8.22385763 14.2238576,8 14.5,8 L15.5,8 C15.7761424,8 16,8.22385763 16,8.5 L16,11 L17.5,11 Z M6,7 C6.55228475,7 7,7.44771525 7,8 C7,8.55228475 6.55228475,9 6,9 L4,9 C3.44771525,9 3,8.55228475 3,8 C3,7.44771525 3.44771525,7 4,7 L6,7 Z M5,13 L3,13 C2.44771525,13 2,12.5522847 2,12 C2,11.4477153 2.44771525,11 3,11 L5,11 C5.55228475,11 6,11.4477153 6,12 C6,12.5522847 5.55228475,13 5,13 Z M6,15 C6.55228475,15 7,15.4477153 7,16 C7,16.5522847 6.55228475,17 6,17 L4,17 C3.44771525,17 3,16.5522847 3,16 C3,15.4477153 3.44771525,15 4,15 L6,15 Z" fill="#2D333F"></path></g></svg></span><span class="_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg">You're in luck! We still have 4 timeslots left</span></div></div><ul class="WYHZZXnzUNBoOX3DRk2bW" data-test="time-slots"><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-0"><a href="https://www.opentable.com/book/validate?rid=1042294&amp;d=2021-08-06T18%3A30%3A00&amp;sd=2021-08-06T18%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=4187390475&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:30 PM Reserve table at REIGN restaurant">6:30 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-1"><a href="https://www.opentable.com/book/validate?rid=1042294&amp;d=2021-08-06T18%3A45%3A00&amp;sd=2021-08-06T18%3A45%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=675581713&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="6:45 PM Reserve table at REIGN restaurant">6:45 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-2"><div class="-Pg4zjVjMUvKsKTST7onV"></div></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-3"><a href="https://www.opentable.com/book/validate?rid=1042294&amp;d=2021-08-06T19%3A15%3A00&amp;sd=2021-08-06T19%3A15%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=3154816886&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:15 PM Reserve table at REIGN restaurant">7:15 PM</a></li><li class="_3SMDUg9wouwZSCQLwzwmtL" data-test="time-slot-4"><a href="https://www.opentable.com/book/validate?rid=1042294&amp;d=2021-08-06T19%3A30%3A00&amp;sd=2021-08-06T19%3A30%3A00&amp;p=2&amp;pt=100&amp;pofids=&amp;hash=395006230&amp;avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJjIjo0LCJzIjowLCJuIjowfQ&amp;corrid=5b5f4267-d324-4de5-92fb-321d57f78f66&amp;tc=default&amp;experienceIds=&amp;ra=&amp;isMandatory=false&amp;st=Standard" class="_3S8tRcYZFFFHDKZowqjtvP" aria-label="7:30 PM Reserve table at REIGN restaurant">7:30 PM</a></li></ul><div class="_35XDbtGx0QfF4BEDjm8mJO _3E4ZQkaXNTgwhWX5Ro2Hu2" data-test="view-dine-out-options-button" role="button" tabindex="0"><p>View dine at home options</p><div class="_3sovCxDbd8zposvTm0dBKC"><span class="_1hUZ3xV2F64jWhaNdNV8RZ _1ON4m3bizfQQCDdsH9NySK" data-test="icDown"><svg viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" focusable="false"><g fill="none" fill-rule="evenodd"><path d="M11,11 L11,14.5 C11,14.7761424 10.7761424,15 10.5,15 L9.5,15 C9.22385763,15 9,14.7761424 9,14.5 L9,10.5 L9,9.5 C9,9.22385763 9.22385763,9 9.5,9 L14.5,9 C14.7761424,9 15,9.22385763 15,9.5 L15,10.5 C15,10.7761424 14.7761424,11 14.5,11 L11,11 Z" fill="#2D333F" transform="translate(12.000000, 12.000000) rotate(-135.000000) translate(-12.000000, -12.000000)"></path></g></svg></span></div></div></div></div></div>

In [ ]:
# //*[@id="mainContent"]/div/div/section[3]/ul/li[2]/div/div[1]/div[1]/a/h6
# <h6 class="k8o46Bca35RzHNtQFy3bH">The Shore Club - Toronto</h6>

In [ ]:
review_df['review-text'][0]

In [ ]:
## how to find action menu to be able to click on later (which i didnt do here)
# soup.find('button', class_='section-review-action-menu')['data-review-id']

In [ ]:
# import re
# nlist = [rev.start() for rev in re.finditer('Response from the owner',soup.text)]
# nlist

In [ ]:
# from selenium.webdriver import ActionChains
# # ActionChains(driver).click('more reviews').perform()
# menu = driver.find_element_by_css_selector('.nav')


In [ ]:
# wait = WebDriverWait(driver, 2)
# menu_bt = wait.until(EC.element_to_be_clickable(
#                        (By.XPATH, '//button[@data-value=\'Sort\']')))


In [ ]:
# ## click on "More Reviews" as many times as needed to load all reviews taken from:
# ## https://github.com/gaspa93/googlemaps-scraper, googlemaps.py
# import time
# links = driver.find_elements_by_xpath('//button[@jsaction=\'pane.reviewChart.moreReviews\']')
# print('LINKS HERE', links)
# for l in links:
#     l.click()
# time.sleep(2)


In [ ]:
# soup = BeautifulSoup(driver.page_source,'html.parser')
# rev_list = soup.find_all('div',class_='section-review-content')
# len(rev_list)

In [ ]:
# import re
# respStart = [rev.start() for rev in re.finditer('Response from the owner',soup.text)]
# respEnd   = [rev.end() for rev in re.finditer('Response from the owner',soup.text)]

In [ ]:
# for ind,r in enumerate(respStart):
#     print(soup.text[r:respEnd[ind]])
#     print('\n')